https://arxiv.org/pdf/2211.10295.pdf

# 1 Initialize

In [ ]:
visible_devices = [1]
import datetime
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing os...")
import os
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing sys...")
import sys
if not any("ipykernel" in arg for arg in sys.argv):
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing and initializing argparse...")
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("-v", "--visible_devices", help="Set visible devices", nargs='*', type=list, default=visible_devices)
    args = parser.parse_args()
    visible_devices = args.visible_devices if args.visible_devices else visible_devices
    if len(visible_devices) == 0:
        visible_devices = int(visible_devices) # type: ignore
    elif len(visible_devices) == 1:
        if len(visible_devices[0]) == 0: # type: ignore
            visible_devices = int(visible_devices[0])
        else:
            visible_devices = [int(i) for i in visible_devices[0]] # type: ignore
    else:
        visible_devices = [int(i) for i in visible_devices]
print("Visible devices:", visible_devices)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timer from timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting env variables for tf import (only device", visible_devices, "will be available)...")
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join([str(i) for i in visible_devices]) # type: ignore
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing numpy...")
import numpy as np
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing matplotlib...")
from matplotlib import pyplot as plt
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing h5py...")
import h5py
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing importlib.util...")
import importlib.util
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing json...")
import json
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing pandas...")
import pandas as pd
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing random...")
import random
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing scipy utils...")
from scipy.stats import norm, lognorm, chi2, kstwo, kstwobign, ks_2samp, moment
from scipy.special import kolmogorov
from scipy.optimize import minimize, curve_fit, root, bisect
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing subprocess...")
import subprocess
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow...")
import tensorflow as tf # type: ignore
print("Tensorflow version:", tf.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tensorflow_probability...")
import tensorflow_probability as tfp # type: ignore
tfd = tfp.distributions
print("Tensorflow probability version:", tfp.__version__)
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing timeit...")
from timeit import default_timer as timer
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing tqdm...")
from tqdm import tqdm
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing typing...")
from typing import List, Tuple, Dict, Callable, Union, Optional, Any, Type
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Setting tf configs...")
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for gpu_device in gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_device, True)

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing GMetrics module...")

sys.path.insert(0,'../utils_func/')
import MixtureDistributions # type: ignore

sys.path.insert(0,'../')
import GMetrics # type: ignore
from GMetrics.utils import se_mean, se_std # type: ignore

print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "Importing jetnet.evaluation.gen_metrics module...")
from jetnet.evaluation import gen_metrics as JMetrics # type: ignore

def get_gpu_info():
    try:
        gpu_info = subprocess.check_output(["nvidia-smi", "--query-gpu=gpu_name", "--format=csv,noheader"]).decode('utf-8')
        return gpu_info.strip().split('\n')
    except Exception as e:
        print(e)
        return None
gpu_models = get_gpu_info()
if gpu_models:
    training_device = gpu_models[eval(os.environ["CUDA_VISIBLE_DEVICES"])]
    print("Successfully loaded GPU model: {}".format(training_device))
else:
    training_device = 'undetermined'
    print("Failed to load GPU model. Defaulting to 'undetermined'.")
    
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")+":", "All done.")

: 

# 2.1 Define distribution

In [2]:
seed = 0
ndims = 10
epsilon = 0.005
GMetrics.utils.reset_random_seeds(seed)
loc1 = np.array(np.zeros(ndims,dtype=np.float64),dtype=np.float64)
scale1 = np.array(np.ones(ndims,dtype=np.float64),dtype=np.float64)
loc2 = np.array(np.random.uniform(-epsilon,epsilon,ndims),dtype=np.float64)
scale2 = np.array(np.random.uniform(1-epsilon,1+epsilon,ndims),dtype=np.float64)
dist_1 = tfd.Independent(tfp.distributions.Normal(loc=loc1, scale=scale1), reinterpreted_batch_ndims=1)
dist_2 = tfd.Independent(tfp.distributions.Normal(loc=loc2, scale=scale2), reinterpreted_batch_ndims=1)
print(dist_1.dtype, dist_2.dtype)
print('Types of distributions: ', type(dist_1), type(dist_2))
print('Dtypes of distributions: ', dist_1.dtype, dist_2.dtype)
print('Batch shapes of distributions: ', dist_1.batch_shape, dist_2.batch_shape)
print('Event shapes of distributions: ', dist_1.event_shape, dist_2.event_shape)

<dtype: 'float64'> <dtype: 'float64'>
Types of distributions:  <class 'tensorflow_probability.python.distributions.independent.Independent'> <class 'tensorflow_probability.python.distributions.independent.Independent'>
Dtypes of distributions:  <dtype: 'float64'> <dtype: 'float64'>
Batch shapes of distributions:  () ()
Event shapes of distributions:  (10,) (10,)


In [3]:
#ncomp = 3
#ndims = 10
#seed = 0
#epsilon = 0.005
#loc1 = np.array(np.zeros([ncomp,ndims],dtype=np.float64),dtype=np.float64)
#scale1 = np.array(np.ones([ncomp,ndims],dtype=np.float64),dtype=np.float64)
#loc2 = np.array(np.random.uniform(-epsilon,epsilon,[ncomp,ndims]),dtype=np.float64)
#scale2 = np.array(np.random.uniform(-epsilon,epsilon,[ncomp,ndims]),dtype=np.float64)
#comp1 = []
#for i in range(ncomp):
#    comp1.append(tfd.MultivariateNormalDiag(loc=loc1[i],scale_diag=scale1[i]))
#comp2 = []
#for i in range(ncomp):
#    comp2.append(tfd.MultivariateNormalDiag(loc=loc2[i],scale_diag=scale2[i]))
#probs = np.random.sample(ncomp)
#dist_1 = tfd.Mixture(
#            cat=tfd.Categorical(probs = probs),
#            components = comp1,
#            validate_args = True)
#dist_2 = tfd.Mixture(
#            cat=tfd.Categorical(probs = probs),
#            components = comp2,
#            validate_args = True)
#print('Types of distributions: ', type(dist_1), type(dist_2))
#print('Dtypes of distributions: ', dist_1.dtype, dist_2.dtype)
#print('Batch shapes of distributions: ', dist_1.batch_shape, dist_2.batch_shape)
#print('Event shapes of distributions: ', dist_1.event_shape, dist_2.event_shape)

# Small samples

## Numpy backend


In [4]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 60,
                                                   batch_size_test = 10_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e7,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = False,
                                                   mirror_strategy = False,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Generating dist_1_num and dist_2_num with tensorflow function.
Checking and setting numerical distributions.
Setting dist_num from dist_symb.


Setting dist_num from dist_symb.
nsamples 600000
batch_size 10000
niter 60
niter * batch_size 600000
small_sample True


In [5]:
print(TwoSampleTestInputs.dist_1_num.shape)
TwoSampleTestInputs.dist_1_num

(600000, 10)


array([[ 0.71485183, -0.10265659, -1.12653662, ...,  0.6962647 ,
        -0.74560825, -0.7065898 ],
       [ 0.32988316, -1.84786602, -1.87617434, ...,  1.87718347,
         1.38581521, -0.27485721],
       [-0.74501223,  0.0191554 , -0.60663805, ..., -0.07774937,
         1.19084498,  0.86549381],
       ...,
       [-1.0699562 ,  0.60049175,  1.63871531, ...,  0.053571  ,
         1.33265117, -0.05276866],
       [-2.06407605, -0.79017413, -0.71623493, ..., -0.04752441,
        -1.47180028,  1.18506031],
       [-0.97621714,  1.18458474,  1.82076853, ...,  1.0020589 ,
        -1.03226671, -0.36630076]])

In [6]:
print(TwoSampleTestInputs.dist_2_num.shape)
TwoSampleTestInputs.dist_2_num

(600000, 10)


array([[-0.45894135, -0.32174056,  0.29579242, ..., -0.70901914,
         1.09032615, -1.18610042],
       [-0.50577605, -1.52055153, -1.08597157, ...,  0.43633271,
         0.63975573,  0.82485925],
       [ 0.43219534, -0.24758369, -0.14955405, ..., -1.18868113,
        -2.09346679,  0.86868804],
       ...,
       [ 0.64125135, -2.38851535, -1.199628  , ..., -1.38971729,
        -1.73347557,  0.8119281 ],
       [-1.6085841 ,  0.29055242,  0.81564289, ...,  2.07250986,
         0.55235748, -0.67259737],
       [ 0.92237169, -0.79592814, -0.30106083, ..., -2.56219424,
         0.30464456,  0.67375019]])

In [7]:
FNMetric = GMetrics.FNMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
FPDMetric = GMetrics.FPDMetric(TwoSampleTestInputs,
                               progress_bar = True,
                               verbose = True)
KPDMetric = GMetrics.KPDMetric(TwoSampleTestInputs,
                               progress_bar = True,
                               verbose = True)
KSTest = GMetrics.KSTest(TwoSampleTestInputs,
                         progress_bar = True,
                         verbose = True)
LRMetric = GMetrics.LRMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
MultiKSTest = GMetrics.MultiKSTest(TwoSampleTestInputs,
                                   progress_bar = True,
                                   verbose = True)
SKSTest = GMetrics.SKSTest(TwoSampleTestInputs,
                           nslices = 100,
                           seed_slicing = 0,
                           progress_bar = True,
                           verbose = True)
SWDMetric = GMetrics.SWDMetric(TwoSampleTestInputs,
                               nslices = 100,
                               seed_slicing = 0,
                               progress_bar = True,
                               verbose = True)

Generating random directions based on nslices, ndims, and seed_slicing.
Generating random directions based on nslices, ndims, and seed_slicing.


### FN Test

In [8]:
FNMetric.Test_np()
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
niter = 10
batch_size = 10000


Iterations: 100%|██████████| 10/10 [00:00<00:00, 644.10it/s]

Running numpy FN calculation...
Two-sample test calculation completed in 0.019357786048203707 seconds.


{'metric_list': array([0.06381937, 0.08351717, 0.06816056, 0.0653649 , 0.06554995,
        0.04301747, 0.08694649, 0.06473251, 0.03918382, 0.05991245])}

### FPD Test

In [14]:
FPDMetric.Test_np()
FPDMetric.Results[-1].result_value


------------------------------------------
Starting FPD metric calculation...
niter = 50
batch_size = 10000


Iterations:   0%|          | 0/50 [00:00<?, ?it/s]

Running numpy FPD calculation...


/local_data/scratch/rtorre/anaconda3/envs/tf2_12/lib/python3.10/site-packages/jetnet/evaluation/gen_metrics.py:692: RuntimeWarning: Recommended number of samples for FPD estimation is 50,000
  warnings.warn("Recommended number of samples for FPD estimation is 50,000", RuntimeWarning)
Iterations: 100%|██████████| 50/50 [01:13<00:00,  1.48s/it]

Two-sample test calculation completed in 73.96762272622436 seconds.


{'metric_list': array([0.00059462, 0.00053794, 0.00040956, 0.00064115, 0.00080315,
        0.00070297, 0.00049225, 0.00053261, 0.00047138, 0.00053657,
        0.00052496, 0.00056957, 0.00052007, 0.00058059, 0.00051315,
        0.00053657, 0.0005367 , 0.00048323, 0.00064603, 0.00047729,
        0.00046198, 0.00067001, 0.00052703, 0.00064577, 0.00060665,
        0.00051659, 0.00061184, 0.00068384, 0.00068339, 0.00070991,
        0.00081032, 0.0005947 , 0.00060446, 0.00087497, 0.00083115,
        0.00055821, 0.00067274, 0.00058977, 0.00070849, 0.00054938,
        0.00057666, 0.00063016, 0.00055271, 0.00054565, 0.00063727,
        0.00049313, 0.00051098, 0.00059691, 0.00063618, 0.00070682]),
 'metric_error_list': array([3.56198377e-05, 5.39869287e-05, 6.74936681e-05, 5.50909118e-05,
        5.87129561e-05, 3.60945411e-05, 4.31497924e-05, 4.32694541e-05,
        3.18269884e-05, 4.41094123e-05, 4.84271857e-05, 4.92312631e-05,
        4.43241476e-05, 3.20262903e-05, 4.68172734e-05, 4.96849608

### KPD Test

In [15]:
KPDMetric.Test_np()
KPDMetric.Results[-1].result_value


------------------------------------------
Starting KPD metric calculation...
niter = 50
batch_size = 10000


Iterations:   0%|          | 0/50 [00:00<?, ?it/s]

Running numpy KPD calculation...


Iterations: 100%|██████████| 50/50 [06:24<00:00,  7.69s/it]

Two-sample test calculation completed in 384.6957740802318 seconds.


{'metric_list': array([ 4.32031227e-05,  8.26072666e-05,  1.22972332e-05,  3.64633557e-05,
         5.02345938e-05,  9.18118753e-05,  1.56355114e-05,  2.10206197e-05,
         6.38994689e-05,  3.49118030e-05,  3.37585205e-05,  7.53087457e-05,
         1.10760533e-05,  1.30421198e-05,  1.60110365e-05,  5.22596257e-05,
         5.81810802e-05,  5.12988683e-05, -3.38207461e-07,  4.17953762e-05,
         5.86508462e-06,  1.90541722e-05,  2.98514123e-05,  1.75193364e-05,
         6.02353326e-05,  2.06102627e-05,  2.07162696e-05,  2.15602384e-05,
         6.72583692e-05,  7.37307378e-05,  3.30799464e-05,  6.07437586e-05,
         1.17971418e-06,  1.16706598e-04,  1.11684587e-04,  1.44129106e-04,
         7.37898218e-05,  3.37431195e-05,  8.02723360e-05,  2.67154935e-05,
         2.03227416e-05,  8.15390440e-05,  3.93504484e-05,  6.10074052e-05,
         2.29975252e-05,  1.64389315e-05,  3.14678058e-05,  6.47265832e-05,
         6.13400822e-05,  8.60206652e-05]),
 'metric_error_list': array([

### KS Test

In [9]:
KSTest.Test_np()
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
niter = 10
batch_size = 10000


Iterations:   0%|          | 0/10 [00:00<?, ?it/s]

Running numpy KS tests...


Iterations: 100%|██████████| 10/10 [00:00<00:00, 45.58it/s]

Two-sample test calculation completed in 0.22124550910666585 seconds.


{'statistic_lists': array([[0.0134, 0.0132, 0.0187, 0.0096, 0.013 ],
        [0.0096, 0.007 , 0.0207, 0.0158, 0.0141],
        [0.0074, 0.0114, 0.0119, 0.0207, 0.0136],
        [0.0076, 0.0155, 0.0091, 0.02  , 0.0156],
        [0.0053, 0.0104, 0.009 , 0.0077, 0.0094],
        [0.0168, 0.0079, 0.009 , 0.0088, 0.0144],
        [0.0122, 0.0154, 0.0121, 0.0083, 0.0149],
        [0.0136, 0.0076, 0.0099, 0.014 , 0.0148],
        [0.0087, 0.0117, 0.0093, 0.0137, 0.0142],
        [0.0082, 0.0123, 0.013 , 0.0085, 0.0174]]),
 'statistic_means': array([0.01358, 0.01344, 0.013  , 0.01356, 0.00836, 0.01138, 0.01258,
        0.01198, 0.01152, 0.01188]),
 'statistic_stds': array([0.00291781, 0.00479441, 0.00435385, 0.00457847, 0.00175795,
        0.00354762, 0.00253093, 0.0027629 , 0.00222926, 0.00337248]),
 'pvalue_lists': array([[0.33054885, 0.34833322, 0.06057885, 0.74616591, 0.36673568],
        [0.74616591, 0.967082  , 0.0275474 , 0.16467352, 0.27321683],
        [0.94711071, 0.53426967, 0.47840

### LR Test

In [9]:
LRMetric.Test_np()
LRMetric.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
niter = 60
batch_size = 10000


Iterations:  10%|█         | 6/60 [00:00<00:00, 56.08it/s]

Running numpy LR calculation...


Iterations: 100%|██████████| 60/60 [00:00<00:00, 141.40it/s]

Two-sample test calculation completed in 0.4295038692653179 seconds.


{'logprob_ref_ref_sum_list': array([-142103.78968538, -142074.16451306, -141806.1244319 ,
        -141814.13429209, -141827.50692153, -141914.78736572,
        -141784.2595698 , -141927.90295817, -141960.80694134,
        -142077.3922252 , -141675.53663609, -141568.6383356 ,
        -142260.01001469, -141755.84508368, -142220.49963952,
        -141848.96993428, -142088.05443493, -141508.33800807,
        -141852.8478124 , -141840.17635951, -142117.5500106 ,
        -141986.83058921, -141409.66825005, -141474.55550333,
        -141580.45816865, -141908.57730732, -142184.56845857,
        -142391.04309727, -141852.1139745 , -141494.573108  ,
        -141669.07523676, -141655.01412501, -141517.79222369,
        -141200.48071813, -141869.82391378, -141907.13832742,
        -142279.49082403, -141513.65514788, -141703.89865503,
        -141731.94133253, -142041.34782309, -141991.17144497,
        -141485.39676367, -142040.2481992 , -141473.24553477,
        -142194.35748645, -141740.18153291

### MultiKS Test

In [11]:
MultiKSTest.Test_np()
MultiKSTest.Results[-1].result_value


------------------------------------------
Starting MultiKS tests calculation...
niter = 10
batch_size = 10000


Iterations:   0%|          | 0/10 [00:00<?, ?it/s]

Running numpy MultiKS tests...


Iterations: 100%|██████████| 10/10 [02:03<00:00, 12.36s/it]

Two-sample test calculation completed in 123.55370635166764 seconds.


{'metric_list': array([0.0163, 0.0186, 0.0203, 0.0233, 0.0152, 0.0154, 0.0199, 0.0168,
        0.0188, 0.0132])}

### SKS Test

In [24]:
SKSTest.Test_np()
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
niter = 10
batch_size = 10000


Iterations:   0%|          | 0/10 [00:00<?, ?it/s]

Running numpy SKS calculation...


Iterations: 100%|██████████| 10/10 [00:05<00:00,  1.73it/s]

Two-sample test calculation completed in 5.788840969093144 seconds.


{'metric_lists': array([[0.0151, 0.0117, 0.0156, 0.009 , 0.0122, 0.0081, 0.009 , 0.0193,
         0.013 , 0.009 , 0.0074, 0.0105, 0.0096, 0.0134, 0.0123, 0.0153,
         0.0148, 0.0148, 0.0145, 0.0142, 0.0154, 0.0087, 0.0081, 0.0184,
         0.0139, 0.0119, 0.0124, 0.0085, 0.0119, 0.0095, 0.0101, 0.0083,
         0.0135, 0.0167, 0.0139, 0.0132, 0.0137, 0.0086, 0.0057, 0.0128,
         0.0148, 0.0127, 0.012 , 0.0081, 0.017 , 0.011 , 0.0133, 0.0074,
         0.0236, 0.013 , 0.0088, 0.0107, 0.0208, 0.0145, 0.0123, 0.0147,
         0.0084, 0.0083, 0.0113, 0.0113, 0.0128, 0.0137, 0.0093, 0.0255,
         0.0147, 0.0176, 0.012 , 0.0082, 0.0109, 0.0077, 0.0095, 0.0159,
         0.0126, 0.0171, 0.0087, 0.0065, 0.0101, 0.0115, 0.01  , 0.0148,
         0.0175, 0.0076, 0.0099, 0.0141, 0.0129, 0.0077, 0.0141, 0.0128,
         0.0093, 0.01  , 0.0116, 0.0143, 0.0165, 0.009 , 0.0066, 0.0151,
         0.0136, 0.0095, 0.0116, 0.0132],
        [0.013 , 0.0091, 0.016 , 0.0083, 0.0087, 0.0095, 0.0159, 0

### SWD Test

In [25]:
SWDMetric.Test_np()
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
niter = 10
batch_size = 10000


Iterations:   0%|          | 0/10 [00:00<?, ?it/s]

Running numpy SKS calculation...


Iterations: 100%|██████████| 10/10 [00:04<00:00,  2.07it/s]

Two-sample test calculation completed in 4.841813870705664 seconds.


{'metric_lists': array([[0.02363572, 0.01778114, 0.03721979, 0.01453548, 0.0176564 ,
         0.01242296, 0.01365917, 0.03079287, 0.02820471, 0.01399335,
         0.01232701, 0.01355767, 0.01639389, 0.0180303 , 0.02227506,
         0.03149092, 0.02306976, 0.02112056, 0.02431377, 0.02881641,
         0.02416798, 0.0173205 , 0.01376633, 0.02714727, 0.02002151,
         0.02100586, 0.02024613, 0.01345072, 0.01771629, 0.01240926,
         0.01556627, 0.01639523, 0.02091818, 0.02445809, 0.02060513,
         0.02695098, 0.01682308, 0.01051256, 0.00874865, 0.02386782,
         0.02231159, 0.02121507, 0.02003033, 0.01176143, 0.02732846,
         0.01379079, 0.02724597, 0.01055707, 0.02948412, 0.01831216,
         0.01017712, 0.01802657, 0.02978577, 0.02704747, 0.01705482,
         0.01874518, 0.01104865, 0.01697128, 0.01687628, 0.02262043,
         0.0203879 , 0.03390408, 0.01666692, 0.03007666, 0.02520135,
         0.02702504, 0.01754665, 0.0092804 , 0.01746313, 0.00949472,
         0.0158780

## TensorFlow backend

In [8]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 10,
                                                   batch_size_test = 10_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e7,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = True,
                                                   mirror_strategy = False,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Checking and setting numerical distributions.
Setting dist_num from dist_symb.
Setting dist_num from dist_symb.
nsamples 100000
batch_size 10000
niter 10
niter * batch_size 100000
small_sample True


In [9]:
TwoSampleTestInputs.dist_1_num

<tf.Tensor: shape=(100000, 5), dtype=float64, numpy=
array([[-2.39967043e-01, -7.94030696e-01,  1.63931164e+00,
         2.71145073e+00, -9.07905520e-01],
       [-7.49834554e-01,  1.67172440e+00,  1.44974904e+00,
        -6.94427146e-01,  1.17355384e-03],
       [ 1.83389833e+00,  2.52622299e-01,  3.59498695e-01,
        -1.24928754e+00, -1.63937696e-01],
       ...,
       [-6.28533146e-01, -1.41448744e+00, -7.68336895e-01,
        -4.48321981e-01,  6.12030513e-01],
       [ 9.82681046e-01, -3.37737389e-01,  8.52866748e-01,
         3.21081079e-01,  1.86262820e+00],
       [-2.65037126e-01, -3.79492339e-01, -5.53834889e-01,
        -5.30263518e-01,  3.42790748e-01]])>

In [10]:
TwoSampleTestInputs.dist_2_num

<tf.Tensor: shape=(100000, 5), dtype=float64, numpy=
array([[-0.5684531 , -2.14725651,  0.06054185, -0.85779686,  1.25558408],
       [ 0.79600364, -0.76710372, -0.79686639,  0.48956836,  0.96199375],
       [-1.11891339, -0.61410463, -0.23228104,  0.18698397, -0.25451839],
       ...,
       [ 1.00934301,  1.83634807,  0.5726704 ,  0.65641066,  0.91162459],
       [ 0.49943041,  1.66088214, -1.06514028,  0.20156471,  0.73097525],
       [-0.92956391, -0.53233223, -0.72287902,  0.46154649, -0.72525675]])>

In [11]:
FNMetric = GMetrics.FNMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
FPDMetric = GMetrics.FPDMetric(TwoSampleTestInputs,
                               progress_bar = True,
                               verbose = True)
KPDMetric = GMetrics.KPDMetric(TwoSampleTestInputs,
                               progress_bar = True,
                               verbose = True)
KSTest = GMetrics.KSTest(TwoSampleTestInputs,
                         progress_bar = True,
                         verbose = True)
LRMetric = GMetrics.LRMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
MultiKSTest = GMetrics.MultiKSTest(TwoSampleTestInputs,
                                   progress_bar = True,
                                   verbose = True)
SKSTest = GMetrics.SKSTest(TwoSampleTestInputs,
                           nslices = 100,
                           seed_slicing = 0,
                           progress_bar = True,
                           verbose = True)
SWDMetric = GMetrics.SWDMetric(TwoSampleTestInputs,
                               nslices = 100,
                               seed_slicing = 0,
                               progress_bar = True,
                               verbose = True)

Generating random directions based on nslices, ndims, and seed_slicing.
Generating random directions based on nslices, ndims, and seed_slicing.


### FN Test

In [30]:
FNMetric.Test_tf(max_vectorize = 2500)
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
Running TF FN calculation...
niter = 10
batch_size = 10000
FN metric calculation completed in 0.7167088030837476 seconds.


{'metric_list': array([0.06381937, 0.08351717, 0.06816056, 0.0653649 , 0.06554995,
        0.04301747, 0.08694649, 0.06473251, 0.03918382, 0.05991245])}

### FPD Test

In [8]:
FPDMetric.Test_tf(max_vectorize = 20)
FPDMetric.Results[-1].result_value


------------------------------------------
Starting FPD metric calculation...
Running TF FPD calculation...
niter = 60
batch_size = 10000
nchunks = 3
Iterating from 0 to 20 out of 60 .


Iterating from 20 to 40 out of 60 .
Iterating from 40 to 60 out of 60 .
vals_list: vals_list=<tf.Tensor: shape=(60, 10), dtype=float64, numpy=
array([[0.00079078, 0.00080683, 0.00080519, 0.00074324, 0.00074207,
        0.00071875, 0.00072294, 0.00071152, 0.00068066, 0.00066755],
       [0.00057942, 0.00050667, 0.00049048, 0.00048334, 0.00045301,
        0.00045257, 0.00042706, 0.00041753, 0.00039084, 0.00038313],
       [0.0005807 , 0.00053657, 0.00053243, 0.00049185, 0.00048038,
        0.00046144, 0.00043615, 0.00043623, 0.00043802, 0.00040788],
       [0.00082954, 0.00075434, 0.00076994, 0.00073501, 0.00072766,
        0.00072561, 0.0006996 , 0.00067678, 0.00068334, 0.00065911],
       [0.0007409 , 0.00068377, 0.00067378, 0.00065856, 0.00065545,
        0.00063149, 0.00066576, 0.00060167, 0.00058003, 0.00057536],
       [0.00073474, 0.00066599, 0.00066843, 0.00062979, 0.00063702,
        0.00061999, 0.00059907, 0.00060262, 0.00053964, 0.00056502],
       [0.0006467 , 0.0006439 , 0.0

{'metric_list': array([0.00068953, 0.00041027, 0.00043178, 0.00067669, 0.00059741,
        0.00057701, 0.00053805, 0.00043665, 0.00082045, 0.00057505,
        0.0005612 , 0.00061483, 0.000455  , 0.0005706 , 0.00049953,
        0.00053008, 0.00071408, 0.00062844, 0.00061231, 0.00062406,
        0.00059482, 0.00063809, 0.00059708, 0.00070794, 0.00058023,
        0.00053743, 0.00061803, 0.00069681, 0.00089431, 0.00061246,
        0.00058261, 0.00061793, 0.00067165, 0.00081573, 0.00080259,
        0.00053762, 0.00053193, 0.00044104, 0.00082616, 0.00059459,
        0.00046533, 0.00054444, 0.00063927, 0.00054115, 0.00049968,
        0.00058352, 0.00055489, 0.0005992 , 0.00064597, 0.00069872,
        0.0009024 , 0.00086008, 0.0005342 , 0.00061873, 0.00051435,
        0.00088234, 0.00062095, 0.00063247, 0.00063875, 0.00057487]),
 'metric_error_list': array([4.53413391e-05, 5.80229676e-05, 5.27344259e-05, 4.73683286e-05,
        4.83596227e-05, 5.39889139e-05, 5.10255942e-05, 4.39177440e-05,
  

### KPD Test

In [15]:
KPDMetric.Test_tf(max_vectorize = 4)
KPDMetric.Results[-1].result_value


------------------------------------------
Starting KPD metric calculation...
Running TF KPD calculation...
niter = 60
batch_size = 10000
nchunks = 15
Iterating from 0 to 4 out of 60 .


Iterating from 4 to 8 out of 60 .
Iterating from 8 to 12 out of 60 .
Iterating from 12 to 16 out of 60 .
Iterating from 16 to 20 out of 60 .
Iterating from 20 to 24 out of 60 .
Iterating from 24 to 28 out of 60 .
Iterating from 28 to 32 out of 60 .
Iterating from 32 to 36 out of 60 .
Iterating from 36 to 40 out of 60 .
Iterating from 40 to 44 out of 60 .
Iterating from 44 to 48 out of 60 .
Iterating from 48 to 52 out of 60 .
Iterating from 52 to 56 out of 60 .
Iterating from 56 to 60 out of 60 .
KPD metric calculation completed in 95.6026419410482 seconds.


{'metric_list': array([ 2.29612695e-05,  1.60385911e-05,  4.48163663e-05,  3.29504022e-05,
         2.08645182e-05,  7.17790097e-05,  3.48015172e-05,  3.61646553e-05,
         1.60612321e-05,  2.20566604e-05,  5.44936862e-05,  7.65324938e-06,
         3.41626924e-05, -2.39888718e-06,  3.34587473e-05,  5.40463178e-05,
         8.02993226e-06, -5.97207557e-06,  4.11701912e-05,  2.01983434e-05,
         7.70303220e-05,  5.98207927e-05,  1.90381188e-05,  3.57092538e-05,
         1.45812784e-05,  6.04014548e-05,  5.73120671e-05,  3.89119511e-05,
         1.33158566e-04,  4.27633793e-05,  4.70900405e-05,  5.39423928e-05,
         2.69892310e-05,  7.25475299e-05,  6.37669137e-05,  6.26878711e-05,
         6.08109782e-05, -2.89705477e-06,  9.03608013e-05, -2.70447736e-06,
         1.01104238e-04,  2.71794759e-05,  3.47233971e-06,  5.38470304e-05,
         2.07921167e-06,  6.57391275e-05,  3.18562188e-05,  1.04847371e-04,
         7.08465478e-05,  1.74792240e-04,  4.07526792e-05,  9.77657025e-0

In [10]:
def _poly_kernel_pairwise(X, Y, degree: int) -> np.ndarray:
    """Pairwise polynomial kernel of degree ``degree`` between X and Y"""
    gamma = 1.0 / X.shape[-1]
    return (X @ Y.T * gamma + 1.0) ** degree
def _mmd_quadratic_unbiased(XX, YY, XY):
    """Calculate quadratic estimate for MMD given pairwise distances between X and Y"""
    m, n = XX.shape[0], YY.shape[0]
    # subtract diagonal 1s
    return (
        (XX.sum() - np.trace(XX)) / (m * (m - 1))
        + (YY.sum() - np.trace(YY)) / (n * (n - 1))
        - 2 * np.mean(XY)
    )
def _mmd_poly_quadratic_unbiased(X, Y, degree: int = 4) -> float:
    """Calculate quadratic estimate for MMD with a polynomial kernel of degree ``degree``"""
    XX = _poly_kernel_pairwise(X, X, degree=degree)
    YY = _poly_kernel_pairwise(Y, Y, degree=degree)
    XY = _poly_kernel_pairwise(X, Y, degree=degree)
    return _mmd_quadratic_unbiased(XX, YY, XY)
def _kpd_batches(X, Y, num_batches, batch_size, seed):
    vals_point = []
    for i in range(num_batches):
        np.random.seed(seed + i * 1_000)
        rand1 = np.random.choice(len(X), size=batch_size)
        rand2 = np.random.choice(len(Y), size=batch_size)

        rand_sample1 = X[rand1]
        rand_sample2 = Y[rand2]

        val = _mmd_poly_quadratic_unbiased(rand_sample1, rand_sample2)
        vals_point.append(val)

    return vals_point

In [12]:
v0 = _kpd_batches(TwoSampleTestInputs.dist_1_num[:10000],TwoSampleTestInputs.dist_1_num[:10000],10,5000,42)
v0

[0.0006625389398742243,
 0.0004542983084911967,
 6.240272517743506e-05,
 -0.0021642069024063737,
 -0.0002327492733860126,
 -1.4299913873916381e-05,
 0.00021013482630083402,
 0.0004080223367259883,
 -0.0015613096410813654,
 5.987234147664111e-05]

In [15]:
np.median(v0)

6.113753332703808e-05

In [13]:
v1 = GMetrics.kpd_metrics.kpd_tf(TwoSampleTestInputs.dist_1_num[:10000],TwoSampleTestInputs.dist_1_num[:10000])
v1

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([ 0.00118308,  0.00050608,  0.00104617, -0.00125501,  0.00183562,
       -0.00025354, -0.00082835, -0.00084631, -0.00035772,  0.00028389])>

In [16]:
np.median(v1)

1.5174629029113262e-05

In [17]:
JMetrics.kpd(TwoSampleTestInputs.dist_1_num[:10000],TwoSampleTestInputs.dist_1_num[:10000])

(-9.067594746836605e-06, 1.4070109379338336e-05)

In [18]:
v = GMetrics.kpd_metrics.kpd_tf(TwoSampleTestInputs.dist_1_num[:10000],TwoSampleTestInputs.dist_1_num[:10000])
v

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([ 0.00024269,  0.00012458, -0.00116677,  0.00086813, -0.00092716,
        0.00047338, -0.00156504, -0.00087922, -0.00024123,  0.00119969])>

In [19]:
GMetrics.kpd_metrics.kpd_tf_output(v)

(array([-5.8325262e-05]), array([0.00087004]))

### KS Test

In [31]:
KSTest.Test_tf(max_vectorize=100)
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 10
batch_size = 10000
KS tests calculation completed in 1.6252719331532717 seconds.


{'statistic_lists': array([[0.01339999, 0.01319999, 0.0187    , 0.00960004, 0.01300001],
        [0.0096    , 0.00700003, 0.02070004, 0.0158    , 0.01409996],
        [0.00740004, 0.01140001, 0.01190001, 0.02070004, 0.01360002],
        [0.00760001, 0.01549999, 0.00909996, 0.02000001, 0.01559997],
        [0.00529999, 0.0104    , 0.009     , 0.00770003, 0.0094    ],
        [0.01680002, 0.0079    , 0.009     , 0.00880003, 0.01440001],
        [0.0122    , 0.01540002, 0.01209998, 0.00830001, 0.01489997],
        [0.01359999, 0.00760001, 0.00990003, 0.014     , 0.01480001],
        [0.00870001, 0.0117    , 0.00930005, 0.01370001, 0.01420003],
        [0.00819999, 0.01230001, 0.01300001, 0.00850004, 0.01740003]]),
 'statistic_means': array([0.01358001, 0.01344   , 0.01300002, 0.01355999, 0.00836   ,
        0.01138001, 0.01258   , 0.01198001, 0.01152002, 0.01188002]),
 'statistic_stds': array([0.0029178 , 0.00479442, 0.00435385, 0.00457848, 0.00175796,
        0.00354762, 0.00253093, 0.00

### LR Test

In [8]:
LRMetric.Test_tf()
LRMetric.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
Running TF LR calculation...
niter = 60
batch_size = 10000


LR metric calculation completed in 1.7027080776169896 seconds.


{'logprob_ref_ref_sum_list': array([-142103.78968538, -142074.16451306, -141806.1244319 ,
        -141814.13429209, -141827.50692153, -141914.78736572,
        -141784.2595698 , -141927.90295817, -141960.80694134,
        -142077.3922252 , -141675.53663609, -141568.6383356 ,
        -142260.01001469, -141755.84508368, -142220.49963952,
        -141848.96993428, -142088.05443493, -141508.33800807,
        -141852.8478124 , -141840.17635951, -142117.5500106 ,
        -141986.83058921, -141409.66825005, -141474.55550333,
        -141580.45816865, -141908.57730732, -142184.56845857,
        -142391.04309727, -141852.1139745 , -141494.573108  ,
        -141669.07523676, -141655.01412501, -141517.79222369,
        -141200.48071813, -141869.82391378, -141907.13832742,
        -142279.49082403, -141513.65514788, -141703.89865503,
        -141731.94133253, -142041.34782309, -141991.17144497,
        -141485.39676367, -142040.2481992 , -141473.24553477,
        -142194.35748645, -141740.18153291

In [32]:
LRMetric.Test_tf()
LRMetric.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
Running TF LR calculation...
niter = 10
batch_size = 10000
LR metric calculation completed in 0.09723280929028988 seconds.


{'logprob_ref_ref_sum_list': array([-71267.65991775, -71192.02693632, -71015.9538473 , -71049.52038285,
        -71260.45818174, -70981.29497924, -70692.02048531, -71049.26794388,
        -71273.8043376 , -70932.77140699]),
 'logprob_ref_alt_sum_list': array([-71093.45584759, -70841.35525502, -70819.6362258 , -70968.46116243,
        -71320.70543361, -71073.89962366, -71212.70552291, -71072.31388476,
        -70944.72723919, -70979.56628662]),
 'logprob_alt_alt_sum_list': array([-71093.13595355, -70840.52723153, -70820.77781157, -70967.01129272,
        -71319.50163863, -71074.22162046, -71213.22432555, -71071.87870667,
        -70943.10785056, -70977.98318687]),
 'lik_ratio_list': array([ 0.63978809,  1.65604698, -2.28317155,  2.89973941,  2.40758997,
        -0.6439936 , -1.03760528,  0.87035618,  3.23877727,  3.1661995 ]),
 'lik_ratio_norm_list': array([ 0.00639788,  0.01656047, -0.02283172,  0.02899739,  0.0240759 ,
        -0.00643994, -0.01037605,  0.00870356,  0.03238777,  0.031

### MultiKS Test

In [33]:
MultiKSTest.Test_tf()
MultiKSTest.Results[-1].result_value


------------------------------------------
Starting MultiKS tests calculation...
Running TF MultiKS tests...
niter = 10
batch_size = 10000
MultiKS tests calculation completed in 0.01761605031788349 seconds.


{'metric_list': array([0.01630002, 0.0186    , 0.0203    , 0.02329999, 0.0152    ,
        0.01540002, 0.01990002, 0.01679999, 0.01879999, 0.0132    ])}

### SKS Test

In [34]:
#tf.config.run_functions_eagerly(True)#
SKSTest.Test_tf(max_vectorize=250)
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
Running TF SKS calculation...
niter = 10
batch_size = 10000
SKS metric calculation completed in 5.812992017250508 seconds.


{'metric_lists': array([[0.0151    , 0.0117    , 0.01559997, 0.009     , 0.0122    ,
         0.00810003, 0.009     , 0.01929998, 0.01300001, 0.009     ,
         0.00740004, 0.01050001, 0.00959998, 0.01340002, 0.01230001,
         0.01530001, 0.01480001, 0.01480001, 0.01450002, 0.01420003,
         0.01539999, 0.00870001, 0.00810003, 0.01840001, 0.01390004,
         0.01190001, 0.0124    , 0.00850001, 0.01190001, 0.00949997,
         0.01010001, 0.00830001, 0.01350001, 0.0167    , 0.01389998,
         0.01320004, 0.01370001, 0.0086    , 0.00570001, 0.01279998,
         0.01480001, 0.01269999, 0.01200002, 0.00810003, 0.01700002,
         0.01100004, 0.0133    , 0.00740001, 0.02360001, 0.01300001,
         0.00879997, 0.01069999, 0.02080002, 0.0145    , 0.01230001,
         0.0147    , 0.00839996, 0.00830001, 0.01130003, 0.0113    ,
         0.01280001, 0.0137    , 0.00929999, 0.0255    , 0.01470001,
         0.0176    , 0.01199999, 0.00820002, 0.01090002, 0.00770003,
         0.0095   

### SWD Test

In [35]:
SWDMetric.Test_tf(max_vectorize=250)
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
Running TF SWD calculation...
niter = 10
batch_size = 10000
SWD metric calculation completed in 1.7945000790059566 seconds.


{'metric_lists': array([[0.02363572, 0.01778114, 0.03721979, 0.01453548, 0.0176564 ,
         0.01242296, 0.01365917, 0.03079287, 0.02820471, 0.01399335,
         0.01232701, 0.01355767, 0.01639389, 0.0180303 , 0.02227506,
         0.03149092, 0.02306976, 0.02112056, 0.02431377, 0.02881641,
         0.02416798, 0.0173205 , 0.01376633, 0.02714727, 0.02002151,
         0.02100586, 0.02024613, 0.01345072, 0.01771629, 0.01240926,
         0.01556627, 0.01639523, 0.02091818, 0.02445809, 0.02060513,
         0.02695098, 0.01682308, 0.01051256, 0.00874865, 0.02386782,
         0.02231159, 0.02121507, 0.02003033, 0.01176143, 0.02732846,
         0.01379079, 0.02724597, 0.01055707, 0.02948412, 0.01831216,
         0.01017712, 0.01802657, 0.02978577, 0.02704747, 0.01705482,
         0.01874518, 0.01104865, 0.01697128, 0.01687628, 0.02262043,
         0.0203879 , 0.03390408, 0.01666692, 0.03007666, 0.02520135,
         0.02702504, 0.01754665, 0.0092804 , 0.01746313, 0.00949472,
         0.0158780

## Jetnet tests

### FPD Test

In [24]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 10,
                                                   batch_size_test = 10_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e7,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = False,
                                                   mirror_strategy = False,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Generating dist_1_num and dist_2_num with tensorflow function.
Checking and setting numerical distributions.
Setting dist_num from dist_symb.
Setting dist_num from dist_symb.
nsamples 100000
batch_size 10000
niter 10
niter * batch_size 100000
small_sample True


In [27]:
d1 = np.array(TwoSampleTestInputs.dist_1_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
d2 = np.array(TwoSampleTestInputs.dist_2_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
print("------------------------------------------")
print("Starting FPD metric calculation...")
print("Running Jetnet FPD calculation...")
print("niter = 10")
print("batch_size = 10000")
start = timer()
FPDResult = [JMetrics.fpd(s1,s2)[0] for s1,s2 in zip(d1,d2)]
end = timer()
print(f"FPD metric calculation completed in {end-start} seconds.")

------------------------------------------
Starting FPD metric calculation...
Running Jetnet FPD calculation...
niter = 10
batch_size = 10000


/local_data/scratch/rtorre/anaconda3/envs/tf2_9/lib/python3.8/site-packages/jetnet/evaluation/gen_metrics.py:698: RuntimeWarning: Recommended number of samples for FPD estimation is 50,000
  warnings.warn("Recommended number of samples for FPD estimation is 50,000", RuntimeWarning)


FPD metric calculation completed in 11.65384515421465 seconds.


In [29]:
FPDResult

[0.00021711883420494262,
 0.0003158271980843811,
 0.00027361061888014936,
 0.00023580432171927144,
 0.00017307851172206515,
 0.00019975293044208664,
 0.000291297030758978,
 0.00023017760328604512,
 0.00018852383969245127,
 0.00017794826133951485]

### KPD Test

In [28]:
d1 = np.array(TwoSampleTestInputs.dist_1_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
d2 = np.array(TwoSampleTestInputs.dist_2_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
print("------------------------------------------")
print("Starting KPD metric calculation...")
print("Running Jetnet KPD calculation...")
print("niter = 10")
print("batch_size = 10000")
start = timer()
KPDResult = [JMetrics.kpd(s1,s2)[0] for s1,s2 in zip(d1,d2)]
end = timer()
print(f"KPD metric calculation completed in {end-start} seconds.")

------------------------------------------
Starting KPD metric calculation...
Running Jetnet KPD calculation...
niter = 10
batch_size = 10000
KPD metric calculation completed in 158.39126108726487 seconds.


In [30]:
KPDResult

[0.10191366092886689,
 0.0799296299077179,
 0.11171410289918526,
 0.1536952109961831,
 -0.17934906441234943,
 -0.1042554083730014,
 0.09455262272575826,
 -0.12348942207734659,
 0.19918233408035757,
 0.15004449810881493]

# Large samples

## Numpy backend


In [3]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 100,
                                                   batch_size_test = 10_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e6,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = False,
                                                   mirror_strategy = False,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Using numpy mode with TensorFlow inputs.
Parsing input distribution...


Input distribution is a tfp.distributions.Distribution object.
Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
nsamples 10000000
batch_size 100000
niter 100
niter * batch_size 10000000
small_sample False


In [6]:
print(TwoSampleTestInputs.dist_1_num.shape)
TwoSampleTestInputs.dist_1_num

(1, 0)


<tf.Tensor: shape=(1, 0), dtype=float64, numpy=array([], shape=(1, 0), dtype=float64)>

In [7]:
print(TwoSampleTestInputs.dist_2_num.shape)
TwoSampleTestInputs.dist_2_num

(1, 0)


<tf.Tensor: shape=(1, 0), dtype=float64, numpy=array([], shape=(1, 0), dtype=float64)>

In [4]:
FNMetric = GMetrics.FNMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
KSTest = GMetrics.KSTest(TwoSampleTestInputs,
                         progress_bar = True,
                         verbose = True)
LRMetric = GMetrics.LRMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
MultiKSTest = GMetrics.MultiKSTest(TwoSampleTestInputs,
                                   progress_bar = True,
                                   verbose = True)
SKSTest = GMetrics.SKSTest(TwoSampleTestInputs,
                           nslices = 100,
                           seed_slicing = 0,
                           progress_bar = True,
                           verbose = True)
SWDMetric = GMetrics.SWDMetric(TwoSampleTestInputs,
                               nslices = 100,
                               seed_slicing = 0,
                               progress_bar = True,
                               verbose = True)

Generating random directions based on nslices, ndims, and seed_slicing.
Generating random directions based on nslices, ndims, and seed_slicing.


### FN Test

In [9]:
FNMetric.Test_np()
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
niter = 100
batch_size = 10000


Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Running numpy FN calculation...


Iterations: 100%|██████████| 100/100 [00:04<00:00, 21.23it/s]

Two-sample test calculation completed in 4.714197907131165 seconds.


{'metric_list': array([0.0712928 , 0.05034113, 0.0468337 , 0.06929792, 0.03731483,
        0.06610887, 0.04393168, 0.0760839 , 0.06789648, 0.07777696,
        0.0717098 , 0.07346497, 0.06765082, 0.05748732, 0.04861635,
        0.0477092 , 0.04121139, 0.05390275, 0.0675732 , 0.03806667,
        0.05139705, 0.06818685, 0.0612675 , 0.07689565, 0.06280929,
        0.05998614, 0.03973218, 0.05506624, 0.05436367, 0.04745789,
        0.06840861, 0.06633603, 0.04886654, 0.05787439, 0.05166639,
        0.05432914, 0.06535938, 0.06245599, 0.04503586, 0.06246507,
        0.05691757, 0.07007181, 0.03513689, 0.06259432, 0.0600962 ,
        0.08274385, 0.05851748, 0.0622261 , 0.04831098, 0.06480682,
        0.07433571, 0.04824319, 0.049485  , 0.05374954, 0.08539418,
        0.05705423, 0.08577949, 0.0693876 , 0.07621036, 0.04625582,
        0.04177584, 0.04041414, 0.0780998 , 0.04188333, 0.06920131,
        0.07086429, 0.04239374, 0.07426549, 0.07664664, 0.09179375,
        0.0442734 , 0.07431389, 0

### KS Test

In [36]:
KSTest.Test_np()
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
niter = 100
batch_size = 10000


Iterations:   3%|▎         | 3/100 [00:00<00:03, 26.59it/s]

Running numpy KS tests...


Iterations: 100%|██████████| 100/100 [00:03<00:00, 26.93it/s]

Two-sample test calculation completed in 3.7165549281053245 seconds.


{'statistic_lists': array([[0.0121, 0.0096, 0.0133, 0.0085, 0.0174],
        [0.0104, 0.0078, 0.0098, 0.0101, 0.0169],
        [0.0139, 0.0073, 0.0076, 0.0136, 0.0139],
        [0.0204, 0.0095, 0.0137, 0.0123, 0.0137],
        [0.0129, 0.0092, 0.0085, 0.0163, 0.013 ],
        [0.0201, 0.0095, 0.0134, 0.0151, 0.009 ],
        [0.0085, 0.0106, 0.0169, 0.0075, 0.016 ],
        [0.0149, 0.0144, 0.0122, 0.0125, 0.0089],
        [0.01  , 0.0101, 0.0092, 0.0129, 0.0134],
        [0.0115, 0.0092, 0.0136, 0.0132, 0.0223],
        [0.0119, 0.009 , 0.0153, 0.0107, 0.0111],
        [0.0094, 0.009 , 0.0107, 0.0138, 0.0125],
        [0.0129, 0.0159, 0.009 , 0.0178, 0.0084],
        [0.0141, 0.007 , 0.0124, 0.0177, 0.0098],
        [0.0112, 0.0178, 0.0202, 0.0104, 0.0109],
        [0.0195, 0.0166, 0.0148, 0.0094, 0.011 ],
        [0.0143, 0.0164, 0.0172, 0.0132, 0.0162],
        [0.0097, 0.0131, 0.005 , 0.0087, 0.0151],
        [0.008 , 0.0231, 0.0151, 0.0142, 0.0075],
        [0.0176, 0.0134, 0.0165

### LR Test

In [37]:
LRMetric.Test_np()
LRMetric.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
niter = 100
batch_size = 10000


Iterations:   5%|▌         | 5/100 [00:00<00:01, 49.26it/s]

Running numpy LR calculation...


Iterations: 100%|██████████| 100/100 [00:01<00:00, 51.74it/s]

Two-sample test calculation completed in 1.9350585862994194 seconds.


{'logprob_ref_ref_sum_list': array([-70901.18122667, -71109.03989312, -71015.39393609, -71006.03789414,
        -70977.27687853, -70762.79690753, -70965.3560591 , -70865.13279837,
        -70884.92874659, -71145.52676865, -70987.5782955 , -71113.34487102,
        -71193.72648664, -70774.97096723, -70973.78528619, -71033.61086275,
        -70971.42001784, -70914.240964  , -70904.00691703, -70919.32166653,
        -70915.69524304, -71059.34569801, -70812.93957552, -71029.09805561,
        -71110.16807105, -70852.56351934, -70948.04069935, -71026.79673571,
        -70692.79964075, -71039.68992471, -70677.90889687, -71099.99126151,
        -71177.13474741, -70960.89028719, -71112.47506746, -70996.15466963,
        -70642.95805024, -71017.44961231, -70846.55178031, -70829.31878489,
        -70919.32562499, -71133.57896337, -70973.45883497, -70700.19468083,
        -70936.78314226, -71052.32182459, -71102.22501891, -71278.69191156,
        -70879.63155939, -70687.01540179, -70735.05749641, -

### MultiKS Test

In [38]:
MultiKSTest.Test_np()
MultiKSTest.Results[-1].result_value


------------------------------------------
Starting MultiKS tests calculation...
niter = 100
batch_size = 10000


Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Running numpy MultiKS tests...


Iterations: 100%|██████████| 100/100 [20:38<00:00, 12.39s/it]

Two-sample test calculation completed in 1238.512193382252 seconds.


{'metric_list': array([0.0205, 0.0178, 0.0328, 0.0149, 0.0208, 0.0161, 0.0165, 0.017 ,
        0.0132, 0.0177, 0.0202, 0.0177, 0.0182, 0.0232, 0.0159, 0.0155,
        0.0209, 0.0153, 0.02  , 0.0178, 0.0154, 0.0168, 0.0182, 0.0133,
        0.0214, 0.0194, 0.0192, 0.0171, 0.0185, 0.0224, 0.0229, 0.0131,
        0.0166, 0.0117, 0.0146, 0.0127, 0.0199, 0.0202, 0.0196, 0.0167,
        0.0179, 0.0184, 0.0151, 0.0179, 0.0186, 0.016 , 0.0145, 0.0213,
        0.0174, 0.0174, 0.0112, 0.0176, 0.0156, 0.0183, 0.0193, 0.0176,
        0.0154, 0.0117, 0.0177, 0.0179, 0.0186, 0.0151, 0.0146, 0.0152,
        0.0181, 0.022 , 0.0264, 0.0166, 0.0215, 0.017 , 0.0153, 0.018 ,
        0.0141, 0.0158, 0.0218, 0.0128, 0.0176, 0.0145, 0.0141, 0.0157,
        0.0152, 0.0191, 0.026 , 0.0159, 0.021 , 0.0156, 0.0229, 0.0133,
        0.0152, 0.0163, 0.0203, 0.016 , 0.0156, 0.0218, 0.0188, 0.016 ,
        0.022 , 0.0139, 0.0225, 0.015 ])}

### SKS Test

In [39]:
SKSTest.Test_np()
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
niter = 100
batch_size = 10000


Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Running numpy SKS calculation...


Iterations: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s]

Two-sample test calculation completed in 57.66074709780514 seconds.


{'metric_lists': array([[0.0222, 0.0136, 0.0115, ..., 0.0153, 0.0105, 0.0221],
        [0.017 , 0.0136, 0.0124, ..., 0.0168, 0.0181, 0.0102],
        [0.0131, 0.0139, 0.0097, ..., 0.0139, 0.0089, 0.0095],
        ...,
        [0.0191, 0.0134, 0.0107, ..., 0.0168, 0.0095, 0.0135],
        [0.019 , 0.0116, 0.0107, ..., 0.012 , 0.0187, 0.0112],
        [0.0152, 0.0066, 0.0149, ..., 0.0068, 0.015 , 0.011 ]]),
 'metric_means': array([0.01509 , 0.013689, 0.011968, 0.01528 , 0.011242, 0.014593,
        0.011454, 0.013672, 0.013098, 0.015856, 0.010627, 0.013476,
        0.013035, 0.017424, 0.012916, 0.014021, 0.012512, 0.011   ,
        0.014715, 0.015032, 0.01141 , 0.010429, 0.012383, 0.012854,
        0.012092, 0.009489, 0.015767, 0.011053, 0.012471, 0.012221,
        0.011641, 0.012558, 0.010192, 0.01232 , 0.013897, 0.013564,
        0.010174, 0.011569, 0.013357, 0.011745, 0.013486, 0.010947,
        0.011327, 0.011515, 0.01264 , 0.01265 , 0.011827, 0.013131,
        0.011788, 0.012412, 0.0

### SWD Test

In [40]:
SWDMetric.Test_np()
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
niter = 100
batch_size = 10000


Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Running numpy SKS calculation...


Iterations: 100%|██████████| 100/100 [00:48<00:00,  2.05it/s]

Two-sample test calculation completed in 48.7373577458784 seconds.


{'metric_lists': array([[0.01551104, 0.01703804, 0.01793974, ..., 0.01827304, 0.01798687,
         0.01173993],
        [0.01129375, 0.01945833, 0.01382552, ..., 0.01836048, 0.01047521,
         0.0129584 ],
        [0.01665323, 0.01572053, 0.01459958, ..., 0.01834031, 0.01270695,
         0.02256029],
        ...,
        [0.01885165, 0.02079276, 0.01417125, ..., 0.02123326, 0.01640981,
         0.01889142],
        [0.02405715, 0.01386171, 0.01865998, ..., 0.01290193, 0.02300509,
         0.01486745],
        [0.01869487, 0.01531223, 0.015929  , ..., 0.01196951, 0.01026871,
         0.01349175]]),
 'metric_means': array([0.01538087, 0.01634466, 0.01602129, 0.02015763, 0.01737095,
        0.01840783, 0.02313696, 0.02078689, 0.01830478, 0.01916848,
        0.0158641 , 0.01464727, 0.01590042, 0.01592523, 0.01879493,
        0.01685413, 0.01710226, 0.01818013, 0.02079662, 0.01502873,
        0.02047189, 0.01970209, 0.02494443, 0.01689681, 0.01811323,
        0.01591965, 0.01665864, 0.017

## TensorFlow backend - w/o MirroredStrategy

In [5]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 100,
                                                   batch_size_test = 100_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e6,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = True,
                                                   mirror_strategy = False,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.


Checking and setting numerical distributions.
Resetting dist_num.
Resetting dist_num.
nsamples 10000000
batch_size 100000
niter 100
niter * batch_size 10000000
small_sample False


In [7]:
TwoSampleTestInputs.dist_1_num

<tf.Tensor: shape=(1, 0), dtype=float64, numpy=array([], shape=(1, 0), dtype=float64)>

In [8]:
TwoSampleTestInputs.dist_2_num

<tf.Tensor: shape=(1, 0), dtype=float64, numpy=array([], shape=(1, 0), dtype=float64)>

In [6]:
FNMetric = GMetrics.FNMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
KSTest = GMetrics.KSTest(TwoSampleTestInputs,
                         progress_bar = True,
                         verbose = True)
LRMetric = GMetrics.LRMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
MultiKSTest = GMetrics.MultiKSTest(TwoSampleTestInputs,
                                   progress_bar = True,
                                   verbose = True)
SKSTest = GMetrics.SKSTest(TwoSampleTestInputs,
                           nslices = 100,
                           seed_slicing = 0,
                           progress_bar = True,
                           verbose = True)
SWDMetric = GMetrics.SWDMetric(TwoSampleTestInputs,
                               nslices = 100,
                               seed_slicing = 0,
                               progress_bar = True,
                               verbose = True)

Generating random directions based on nslices, ndims, and seed_slicing.
Generating random directions based on nslices, ndims, and seed_slicing.


### FN Test

In [19]:
FNMetric.Test_tf(max_vectorize = 100)
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
Running TF FN calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
FN metric calculation completed in 6.2151684099808335 seconds.


{'metric_list': array([0.08844678, 0.06461692, 0.06138989, 0.0570237 , 0.05180524,
        0.03899186, 0.0806584 , 0.05132419, 0.05959165, 0.0743582 ,
        0.04163835, 0.06541973, 0.0836046 , 0.06609202, 0.07858423,
        0.07299963, 0.08009843, 0.06226545, 0.07964515, 0.02545401,
        0.06326744, 0.06652458, 0.06568475, 0.04437675, 0.04462649,
        0.05019201, 0.04662913, 0.0695184 , 0.05447623, 0.07051035,
        0.05697775, 0.09103571, 0.03763976, 0.06669034, 0.05780751,
        0.06299498, 0.05586779, 0.08759443, 0.04460861, 0.0781111 ,
        0.05357439, 0.07744134, 0.07074611, 0.07235557, 0.06637625,
        0.07226581, 0.03865412, 0.05803577, 0.07475699, 0.05028286,
        0.09106504, 0.0576629 , 0.04348506, 0.06311519, 0.06380644,
        0.05714464, 0.0678285 , 0.0451138 , 0.03682507, 0.04246485,
        0.06364599, 0.07168154, 0.06391398, 0.07486832, 0.08324359,
        0.05073289, 0.06336298, 0.05440357, 0.06233026, 0.04763888,
        0.07412372, 0.06897391, 0

In [17]:
FNMetric.Test_tf(max_vectorize = 20)
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
Running TF FN calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
FN metric calculation completed in 8.154968474060297 seconds.


{'metric_list': array([0.06589649, 0.05694527, 0.07066927, 0.05432756, 0.04851787,
        0.05148932, 0.05831022, 0.06762085, 0.06135772, 0.05813442,
        0.04395214, 0.07256788, 0.04990327, 0.07285298, 0.07490167,
        0.07542354, 0.05599153, 0.0652228 , 0.0782739 , 0.06864965,
        0.06071743, 0.07208004, 0.05251999, 0.05203989, 0.05447789,
        0.06208448, 0.04080829, 0.06393315, 0.04100564, 0.05381943,
        0.08149364, 0.06090667, 0.05081972, 0.06528786, 0.06153516,
        0.07378813, 0.04698807, 0.05587642, 0.03221431, 0.05341343,
        0.05793074, 0.06182036, 0.05764185, 0.07427764, 0.04222705,
        0.04352648, 0.05651465, 0.06179742, 0.03603826, 0.06871026,
        0.04996418, 0.05690464, 0.06886232, 0.08153194, 0.05562752,
        0.03799642, 0.08106984, 0.05613789, 0.07105397, 0.03500809,
        0.05062465, 0.06815482, 0.06846427, 0.07182132, 0.08157124,
        0.07486225, 0.04875092, 0.03786518, 0.04123019, 0.05824449,
        0.06169303, 0.07860587, 0

### KS Test

In [22]:
KSTest.Test_tf(max_vectorize=1000)
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
KS tests calculation completed in 7.201745927333832 seconds.


{'statistic_lists': array([[0.01539999, 0.01589996, 0.01350001, 0.0113    , 0.01520002],
        [0.00949997, 0.0126    , 0.00929999, 0.0176    , 0.01010001],
        [0.01010001, 0.009     , 0.01240003, 0.01090002, 0.01460004],
        [0.00820005, 0.0108    , 0.01499999, 0.0114    , 0.01200002],
        [0.01090002, 0.01160002, 0.00579999, 0.01109999, 0.0105    ],
        [0.00940001, 0.01390004, 0.01010001, 0.01390004, 0.01230001],
        [0.01550001, 0.00989997, 0.01100001, 0.01369998, 0.01019999],
        [0.01459998, 0.01089996, 0.01010001, 0.0125    , 0.01490003],
        [0.01660001, 0.01550001, 0.009     , 0.00649995, 0.0122    ],
        [0.02420002, 0.00840002, 0.0057    , 0.0088    , 0.01109999],
        [0.00960001, 0.0097    , 0.0151    , 0.01100001, 0.01199999],
        [0.00819999, 0.00870001, 0.01419997, 0.01549999, 0.01390001],
        [0.01710001, 0.0098    , 0.01609999, 0.0149    , 0.0106    ],
        [0.0091    , 0.01620001, 0.01280004, 0.023     , 0.00989999],
 

In [23]:
KSTest.Test_tf(max_vectorize=100)
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
KS tests calculation completed in 12.036879192106426 seconds.


{'statistic_lists': array([[0.01359999, 0.0134    , 0.00869998, 0.0086    , 0.01429999],
        [0.01529998, 0.00940001, 0.01100004, 0.0059    , 0.01100001],
        [0.01530004, 0.01320004, 0.009     , 0.00810003, 0.01950002],
        [0.00850004, 0.013     , 0.01880002, 0.02599999, 0.01430005],
        [0.01289999, 0.01119998, 0.00759999, 0.00929999, 0.0102    ],
        [0.01199999, 0.0079    , 0.00879997, 0.01030001, 0.0088    ],
        [0.02410001, 0.01199999, 0.01029998, 0.00919998, 0.01169997],
        [0.01180005, 0.00600004, 0.00890002, 0.0122    , 0.0108    ],
        [0.02320004, 0.016     , 0.0086    , 0.00979996, 0.01809999],
        [0.01210004, 0.0079    , 0.0158    , 0.00740004, 0.0106    ],
        [0.00980002, 0.0079    , 0.01770002, 0.00719994, 0.01390004],
        [0.00730002, 0.01139998, 0.01540005, 0.0169    , 0.01539999],
        [0.01800001, 0.0108    , 0.01880002, 0.01789999, 0.0116    ],
        [0.01409999, 0.00820005, 0.0086    , 0.0156    , 0.0108    ],
 

### LR Test

In [24]:
LRMetric.Test_tf()
LRMetric.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
Running TF LR calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
LR metric calculation completed in 5.55027876701206 seconds.


{'logprob_ref_ref_sum_list': array([-71267.65991775, -71015.9538473 , -71260.45818174, -70692.02048531,
        -71273.8043376 , -71011.10317567, -70738.7450337 , -71237.46886075,
        -71131.19154768, -70861.07507269, -71023.39914442, -70788.25762199,
        -71056.5066197 , -71238.97805322, -71022.1250929 , -70979.52058058,
        -71021.5879943 , -70943.91075143, -71014.07676012, -71018.51654553,
        -71031.15775373, -70624.50172146, -70745.20551332, -70899.51048067,
        -70890.6922343 , -71102.44868384, -71124.82393212, -71120.6706772 ,
        -71126.86352794, -70962.40294566, -71286.00158211, -71016.75517526,
        -70986.71582069, -70778.38301096, -70838.44166557, -70698.22736843,
        -70983.7670851 , -70909.87760942, -71030.3358148 , -71147.44525311,
        -71078.9280889 , -71013.62444443, -70980.78376952, -70876.21199649,
        -71004.08662644, -70813.82788588, -71108.00697788, -71060.63809415,
        -71006.61535766, -70927.18594706, -70928.04708992, -

### MultiKS Test

In [7]:
MultiKSTest.Test_tf()
MultiKSTest.Results[-1].result_value


------------------------------------------
Starting MultiKS tests calculation...
Running TF MultiKS tests...
niter = 100
batch_size = 100000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.


### SKS Test

In [8]:
#tf.config.run_functions_eagerly(True)#
SKSTest.Test_tf(max_vectorize=100)
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
Running TF SKS calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
SKS metric calculation completed in 21.715895311906934 seconds.


{'metric_lists': array([[0.01480001, 0.0081    , 0.02020001, ..., 0.00940001, 0.01130003,
         0.01370001],
        [0.01749998, 0.00820005, 0.01180001, ..., 0.00749999, 0.00980002,
         0.01990002],
        [0.0126    , 0.00770003, 0.00690001, ..., 0.00929999, 0.01050001,
         0.00749999],
        ...,
        [0.01059997, 0.0104    , 0.01269996, ..., 0.008     , 0.00910002,
         0.01250002],
        [0.00620002, 0.01069999, 0.01090002, ..., 0.0117    , 0.0115    ,
         0.0068    ],
        [0.00850001, 0.00849999, 0.01350003, ..., 0.0104    , 0.00850004,
         0.011     ]]),
 'metric_means': array([0.013364  , 0.01318601, 0.010608  , 0.01182601, 0.010928  ,
        0.01169   , 0.01253501, 0.010403  , 0.01191501, 0.01304801,
        0.01092601, 0.01294501, 0.01307001, 0.01467801, 0.012637  ,
        0.01331901, 0.011416  , 0.012247  , 0.01134401, 0.01031401,
        0.01228901, 0.011523  , 0.012047  , 0.014114  , 0.01319401,
        0.010371  , 0.01294501, 0.011

In [9]:
#tf.config.run_functions_eagerly(True)#
SKSTest.Test_tf(max_vectorize=20)
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
Running TF SKS calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
SKS metric calculation completed in 37.79218263179064 seconds.


{'metric_lists': array([[0.01300001, 0.02380002, 0.01480001, ..., 0.01650003, 0.01320004,
         0.01280001],
        [0.01490003, 0.0147    , 0.00690001, ..., 0.014     , 0.00620002,
         0.00929999],
        [0.01560003, 0.02020001, 0.0133    , ..., 0.0147    , 0.01720001,
         0.00909999],
        ...,
        [0.00920001, 0.0086    , 0.01780003, ..., 0.0134    , 0.0126    ,
         0.0194    ],
        [0.01020002, 0.01820004, 0.02340001, ..., 0.01450002, 0.01920001,
         0.0108    ],
        [0.00929999, 0.00979996, 0.0112    , ..., 0.00690001, 0.01749998,
         0.0079    ]]),
 'metric_means': array([0.01275901, 0.010881  , 0.01321701, 0.014297  , 0.011023  ,
        0.01068801, 0.012679  , 0.01009201, 0.013536  , 0.011056  ,
        0.01144001, 0.01325601, 0.01402801, 0.01183201, 0.01072601,
        0.014323  , 0.013275  , 0.011977  , 0.01152601, 0.01221601,
        0.01359301, 0.015167  , 0.01141901, 0.01151   , 0.011964  ,
        0.01425701, 0.01252001, 0.011

### SWD Test

In [10]:
SWDMetric.Test_tf(max_vectorize=100)
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
Running TF SWD calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
SWD metric calculation completed in 12.050446098670363 seconds.


{'metric_lists': array([[0.03162819, 0.01378654, 0.04088677, ..., 0.01345777, 0.0194596 ,
         0.0216528 ],
        [0.02483566, 0.00997102, 0.01615505, ..., 0.01193278, 0.01140177,
         0.02853671],
        [0.0173473 , 0.01196029, 0.00882089, ..., 0.00979251, 0.01519314,
         0.01147516],
        ...,
        [0.01455991, 0.01299323, 0.01789068, ..., 0.01131896, 0.01421481,
         0.02169172],
        [0.00881808, 0.01586686, 0.01922807, ..., 0.01472376, 0.01762778,
         0.01080257],
        [0.01468322, 0.01230006, 0.01595806, ..., 0.01449617, 0.01777882,
         0.02153003]]),
 'metric_means': array([0.02305994, 0.01961933, 0.01494935, 0.0165957 , 0.01540153,
        0.01630492, 0.01844658, 0.01457396, 0.01850644, 0.01928812,
        0.01594006, 0.02060427, 0.02165779, 0.02243471, 0.01922148,
        0.01927906, 0.01618883, 0.01797778, 0.01694674, 0.0133173 ,
        0.01801064, 0.01667891, 0.01798509, 0.0216246 , 0.02078888,
        0.01405203, 0.01992054, 0.016

In [11]:
SWDMetric.Test_tf(max_vectorize=20)
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
Running TF SWD calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
SWD metric calculation completed in 16.791635151021183 seconds.


{'metric_lists': array([[0.01696587, 0.02975092, 0.02688201, ..., 0.02597215, 0.0236831 ,
         0.02127402],
        [0.02650608, 0.01575183, 0.01021158, ..., 0.01528896, 0.01000004,
         0.01347874],
        [0.0209232 , 0.03094118, 0.02536397, ..., 0.02210008, 0.03162633,
         0.01020194],
        ...,
        [0.01723425, 0.01339058, 0.02342225, ..., 0.01498974, 0.01863742,
         0.02494219],
        [0.01238241, 0.03199633, 0.03446509, ..., 0.03018332, 0.02660104,
         0.01802572],
        [0.01440825, 0.0148892 , 0.02190768, ..., 0.01191863, 0.02190961,
         0.01231276]]),
 'metric_means': array([0.01950023, 0.01492762, 0.02101212, 0.02384391, 0.015534  ,
        0.01536133, 0.01964435, 0.0141888 , 0.02125429, 0.01662639,
        0.01604551, 0.01992048, 0.02194673, 0.01654004, 0.01568991,
        0.02131496, 0.01978949, 0.01712843, 0.01735812, 0.01763901,
        0.02116491, 0.02315418, 0.01630083, 0.0159796 , 0.0177388 ,
        0.0212523 , 0.01761649, 0.016

## TensorFlow backend - w/ MirroredStrategy

In [12]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 100,
                                                   batch_size_test = 10_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e6,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = True,
                                                   mirror_strategy = True,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Checking and setting numerical distributions.
Resetting dist_num.
Resetting dist_num.
nsamples 1000000
batch_size 10000
niter 100
niter * batch_size 1000000
small_sample False


In [13]:
TwoSampleTestInputs.dist_1_num

<tf.Tensor: shape=(1, 0), dtype=float64, numpy=array([], shape=(1, 0), dtype=float64)>

In [14]:
TwoSampleTestInputs.dist_2_num

<tf.Tensor: shape=(1, 0), dtype=float64, numpy=array([], shape=(1, 0), dtype=float64)>

In [15]:
FNMetric = GMetrics.FNMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
KSTest = GMetrics.KSTest(TwoSampleTestInputs,
                         progress_bar = True,
                         verbose = True)
LRMetric = GMetrics.LRMetric(TwoSampleTestInputs,
                             progress_bar = True,
                             verbose = True)
MultiKSTest = GMetrics.MultiKSTest(TwoSampleTestInputs,
                                   progress_bar = True,
                                   verbose = True)
SKSTest = GMetrics.SKSTest(TwoSampleTestInputs,
                           nslices = 100,
                           seed_slicing = 0,
                           progress_bar = True,
                           verbose = True)
SWDMetric = GMetrics.SWDMetric(TwoSampleTestInputs,
                               nslices = 100,
                               seed_slicing = 0,
                               progress_bar = True,
                               verbose = True)

Generating random directions based on nslices, ndims, and seed_slicing.
Generating random directions based on nslices, ndims, and seed_slicing.


### FN Test

In [16]:
FNMetric.Test_tf(max_vectorize = 100)
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
Running TF FN calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
FN metric calculation completed in 21.112920443993062 seconds.


{'metric_list': array([0.05516948, 0.07523884, 0.05139113, 0.06802936, 0.04490469,
        0.05978485, 0.0455044 , 0.07092618, 0.05965368, 0.07004246,
        0.06199579, 0.074203  , 0.07081407, 0.06162242, 0.05763099,
        0.04878653, 0.0440177 , 0.07121686, 0.05928487, 0.05353271,
        0.05007546, 0.06046232, 0.06235922, 0.0537009 , 0.05698305,
        0.04858227, 0.07860934, 0.08878225, 0.06568089, 0.08181209,
        0.03826476, 0.04858537, 0.07847674, 0.09424529, 0.06727678,
        0.0746056 , 0.07498207, 0.04279251, 0.06313507, 0.0536917 ,
        0.06171695, 0.05284828, 0.06473213, 0.07048272, 0.064618  ,
        0.05713674, 0.08083268, 0.03683843, 0.06893184, 0.08803626,
        0.0492056 , 0.04397483, 0.05598315, 0.05087937, 0.08014898,
        0.03877202, 0.0836444 , 0.04161174, 0.06967698, 0.06442132,
        0.07953344, 0.0737494 , 0.08114015, 0.08533532, 0.06960615,
        0.0676165 , 0.06053766, 0.05043386, 0.068296  , 0.05128079,
        0.05139539, 0.05998076, 0

In [17]:
FNMetric.Test_tf(max_vectorize = 20)
FNMetric.Results[-1].result_value


------------------------------------------
Starting FN metric calculation...
Running TF FN calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
FN metric calculation completed in 22.46244783513248 seconds.


{'metric_list': array([0.05452402, 0.05304218, 0.04884497, 0.05827171, 0.06813584,
        0.0578599 , 0.0655281 , 0.05902291, 0.0758386 , 0.07137658,
        0.06437424, 0.06101667, 0.05150613, 0.07834832, 0.04646586,
        0.03409809, 0.06332212, 0.04656546, 0.05117588, 0.05383656,
        0.07197195, 0.05900689, 0.09463766, 0.07333664, 0.06263213,
        0.07135769, 0.073785  , 0.05501361, 0.06074989, 0.05249054,
        0.07000258, 0.04532946, 0.05612702, 0.05117352, 0.06257506,
        0.06356542, 0.09491339, 0.06699292, 0.04492981, 0.09103823,
        0.04302169, 0.09187906, 0.0659404 , 0.07321564, 0.059644  ,
        0.05031904, 0.07439416, 0.08166692, 0.08396725, 0.05272624,
        0.06706975, 0.03746853, 0.06315798, 0.06965421, 0.06656968,
        0.05518958, 0.07083439, 0.07465477, 0.03838982, 0.05963312,
        0.04399156, 0.05031205, 0.05306584, 0.05351291, 0.07989406,
        0.06935564, 0.04512011, 0.05883892, 0.06931485, 0.0589221 ,
        0.0450616 , 0.05494645, 0

### KS Test

In [18]:
KSTest.Test_tf(max_vectorize=1000)
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
KS tests calculation completed in 20.85198868997395 seconds.


{'statistic_lists': array([[0.01480001, 0.01210001, 0.01050001, 0.00950003, 0.00639999],
        [0.00929999, 0.01410002, 0.01230001, 0.0107    , 0.0091    ],
        [0.01199999, 0.02060002, 0.00950001, 0.01550001, 0.01269999],
        [0.00570002, 0.01570001, 0.01210001, 0.01270001, 0.01709998],
        [0.01339996, 0.00849999, 0.01600003, 0.01249999, 0.01370001],
        [0.01159999, 0.0141    , 0.0086    , 0.01110002, 0.01109999],
        [0.01100001, 0.00850001, 0.01370001, 0.01009999, 0.01010001],
        [0.01090001, 0.01200002, 0.01090002, 0.01350001, 0.01020002],
        [0.00860003, 0.00910002, 0.01750001, 0.00989999, 0.01100004],
        [0.01300001, 0.01289999, 0.01389998, 0.00870001, 0.01350001],
        [0.00890002, 0.01060003, 0.0097    , 0.01500005, 0.01420003],
        [0.01440001, 0.0104    , 0.01340002, 0.0122    , 0.0113    ],
        [0.00930002, 0.01639998, 0.01000005, 0.01530004, 0.00710005],
        [0.01160002, 0.0088    , 0.01350003, 0.01449999, 0.0088    ],
 

In [19]:
KSTest.Test_tf(max_vectorize=100)
KSTest.Results[-1].result_value


------------------------------------------
Starting KS tests calculation...
Running TF KS tests...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
KS tests calculation completed in 26.09462933195755 seconds.


{'statistic_lists': array([[0.0087    , 0.02129999, 0.01590002, 0.01050001, 0.00800002],
        [0.01720005, 0.00760001, 0.0122    , 0.00630003, 0.01280004],
        [0.0158    , 0.01599997, 0.01480001, 0.01780003, 0.01309997],
        [0.01029998, 0.0108    , 0.02240002, 0.01480001, 0.01189999],
        [0.01109999, 0.01539999, 0.0108    , 0.01209998, 0.01499999],
        [0.01189995, 0.00800002, 0.0138    , 0.01250002, 0.0244    ],
        [0.0097    , 0.01230001, 0.00930002, 0.01960003, 0.01190001],
        [0.01559997, 0.00729999, 0.00840002, 0.014     , 0.01160002],
        [0.01240003, 0.01340002, 0.0148    , 0.01029998, 0.01470006],
        [0.0106    , 0.01530001, 0.0196    , 0.01050001, 0.0133    ],
        [0.01440001, 0.0114    , 0.0156    , 0.01000005, 0.01539999],
        [0.01449996, 0.01210001, 0.02120003, 0.01400001, 0.00739998],
        [0.00909999, 0.01170003, 0.01109999, 0.01100001, 0.01450002],
        [0.01320001, 0.01120001, 0.01590002, 0.0088    , 0.0176    ],
 

### LR Test

In [20]:
LRMetric.Test_tf()
LRMetric.Results[-1].result_value


------------------------------------------
Starting LR metric calculation...
Running TF LR calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
LR metric calculation completed in 16.49125451222062 seconds.


{'logprob_ref_ref_sum_list': array([-71122.35778678, -70997.08311632, -70945.30444864, -71212.53145864,
        -71023.39321228, -70881.96161133, -70850.4958714 , -70833.69011229,
        -71158.36375992, -70979.60110375, -70781.70154756, -70633.07070525,
        -71171.07183999, -70964.73030259, -70993.16958503, -70966.18715412,
        -70868.10813108, -71000.55827826, -71015.19665669, -70886.08186536,
        -71068.44044019, -71133.5133569 , -71145.6357901 , -70999.88559598,
        -71060.72781329, -70811.20784462, -70891.12994454, -70921.37040776,
        -70944.9896813 , -70825.11917232, -70949.92511443, -71018.55673099,
        -70832.88249438, -70779.90665855, -71007.33855508, -71147.37536696,
        -70938.48282797, -70809.46691485, -71088.46522738, -71065.84795579,
        -70740.43992664, -71274.98592255, -71025.25252887, -70668.13540263,
        -70795.27884184, -71157.72379093, -70700.78734079, -70946.15733742,
        -70866.95802175, -70937.8908087 , -70907.48809808, -

### MultiKS Test

In [21]:
MultiKSTest.Test_tf()
MultiKSTest.Results[-1].result_value


------------------------------------------
Starting MultiKS tests calculation...
Running TF MultiKS tests...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MultiKS tests calculation completed in 19.153131428174675 seconds.


{'metric_list': array([0.0141    , 0.0143    , 0.01449999, 0.01389998, 0.0149    ,
        0.0176    , 0.0126    , 0.0176    , 0.01449999, 0.0131    ,
        0.02429999, 0.02290004, 0.0167    , 0.01910001, 0.0187    ,
        0.0202    , 0.01679999, 0.0156    , 0.01769999, 0.0124    ,
        0.0149    , 0.0201    , 0.021     , 0.02200001, 0.01879999,
        0.01970001, 0.0131    , 0.021     , 0.02069998, 0.01820001,
        0.0141    , 0.01639998, 0.0161    , 0.01429999, 0.02180001,
        0.01430002, 0.01860002, 0.01500002, 0.01930001, 0.01640001,
        0.0192    , 0.01680002, 0.02000001, 0.01989999, 0.01980001,
        0.01410002, 0.02219999, 0.0214    , 0.0223    , 0.01700002,
        0.01800001, 0.01679999, 0.01699999, 0.0196    , 0.0123    ,
        0.01429999, 0.02070004, 0.02590001, 0.0151    , 0.02219999,
        0.01570001, 0.01800001, 0.01710001, 0.01659998, 0.02039999,
        0.01990002, 0.0219    , 0.01530001, 0.01569999, 0.016     ,
        0.0191    , 0.01609999, 0

### SKS Test

In [22]:
#tf.config.run_functions_eagerly(True)#
SKSTest.Test_tf(max_vectorize=100)
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
Running TF SKS calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
SKS metric calculation completed in 31.295621445868164 seconds.


{'metric_lists': array([[0.01109999, 0.00939998, 0.0081    , ..., 0.01280004, 0.00680003,
         0.0124    ],
        [0.00840002, 0.00829998, 0.01440001, ..., 0.00710002, 0.01669997,
         0.0079    ],
        [0.0126    , 0.01319999, 0.009     , ..., 0.0072    , 0.02330005,
         0.0131    ],
        ...,
        [0.01240003, 0.0094    , 0.0132    , ..., 0.00939998, 0.01429999,
         0.00800002],
        [0.0104    , 0.01369998, 0.0115    , ..., 0.0183    , 0.01210001,
         0.01339999],
        [0.00749999, 0.0115    , 0.00759998, ..., 0.01030004, 0.0122    ,
         0.00739998]]),
 'metric_means': array([0.01237401, 0.011442  , 0.01320601, 0.012035  , 0.01171801,
        0.01117901, 0.011078  , 0.012075  , 0.011681  , 0.01228501,
        0.012431  , 0.012968  , 0.01249601, 0.01034801, 0.01393901,
        0.013564  , 0.009303  , 0.011152  , 0.01220001, 0.013155  ,
        0.012076  , 0.013315  , 0.01191501, 0.01191401, 0.01241801,
        0.01158601, 0.01299501, 0.011

In [23]:
#tf.config.run_functions_eagerly(True)#
SKSTest.Test_tf(max_vectorize=20)
SKSTest.Results[-1].result_value


------------------------------------------
Starting SKS metric calculation...
Running TF SKS calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
SKS metric calculation completed in 51.64437641389668 seconds.


{'metric_lists': array([[0.0101    , 0.01069999, 0.01920003, ..., 0.01560003, 0.01090002,
         0.01129997],
        [0.01610005, 0.01209998, 0.00979999, ..., 0.0159    , 0.01470003,
         0.009     ],
        [0.01020002, 0.01109998, 0.0138    , ..., 0.0117    , 0.02350003,
         0.01220001],
        ...,
        [0.01169997, 0.0122    , 0.01100001, ..., 0.0104    , 0.01090002,
         0.00940001],
        [0.00830001, 0.01410002, 0.0081    , ..., 0.00670004, 0.01319999,
         0.0115    ],
        [0.00889999, 0.0115    , 0.00569999, ..., 0.0122    , 0.00739998,
         0.00910002]]),
 'metric_means': array([0.01327   , 0.01190101, 0.01469301, 0.01079   , 0.01169101,
        0.01345501, 0.01372101, 0.012122  , 0.01181   , 0.013218  ,
        0.01265801, 0.01399501, 0.011882  , 0.01241601, 0.01292401,
        0.01299501, 0.01189401, 0.009822  , 0.012706  , 0.011539  ,
        0.01081501, 0.01235601, 0.01142301, 0.01133101, 0.01235301,
        0.010625  , 0.01368801, 0.012

### SWD Test

In [24]:
SWDMetric.Test_tf(max_vectorize=100)
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
Running TF SWD calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
SWD metric calculation completed in 25.397612144704908 seconds.


{'metric_lists': array([[0.02109074, 0.01611311, 0.01444374, ..., 0.01855509, 0.01239889,
         0.01680351],
        [0.01380003, 0.01042606, 0.02202222, ..., 0.00932223, 0.0177067 ,
         0.01147555],
        [0.02416267, 0.01218935, 0.01381469, ..., 0.01091553, 0.03472535,
         0.01949777],
        ...,
        [0.01539123, 0.0146195 , 0.01641249, ..., 0.01569456, 0.01899798,
         0.0104228 ],
        [0.01658863, 0.02575276, 0.02136341, ..., 0.02866213, 0.01545548,
         0.01932009],
        [0.01099984, 0.01596476, 0.00999818, ..., 0.01277605, 0.02150151,
         0.01020842]]),
 'metric_means': array([0.019023  , 0.01716178, 0.02024052, 0.01634261, 0.01633826,
        0.01629205, 0.01595578, 0.01786084, 0.01660539, 0.01775773,
        0.01841084, 0.01978751, 0.01909857, 0.01492454, 0.02289222,
        0.01946847, 0.01298747, 0.01661376, 0.01790238, 0.0195786 ,
        0.018306  , 0.02129012, 0.01847467, 0.01808385, 0.01857158,
        0.018253  , 0.02056439, 0.017

In [25]:
SWDMetric.Test_tf(max_vectorize=20)
SWDMetric.Results[-1].result_value


------------------------------------------
Starting SWD metric calculation...
Running TF SWD calculation...
niter = 100
batch_size = 10000
The dist_1_num tensor is empty. Batches will be generated 'on-the-fly' from dist_1_symb.
The dist_2_num tensor is empty. Batches will be generated 'on-the-fly' from dist_2_symb.
nchunks = 5
Iterating from 0 to 20 out of 100 .
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Iterating from 20 to 40 out of 100 .
Iterating from 40 to 60 out of 100 .
Iterating from 60 to 80 out of 100 .
Iterating from 80 to 100 out of 100 .
SWD metric calculation completed in 30.01362854614854 seconds.


{'metric_lists': array([[0.01942781, 0.02088098, 0.03387376, ..., 0.02185463, 0.01329031,
         0.01644927],
        [0.02353514, 0.01864871, 0.01136168, ..., 0.02218574, 0.02041629,
         0.01441108],
        [0.01553319, 0.01823029, 0.01724589, ..., 0.01905348, 0.03161396,
         0.02151386],
        ...,
        [0.01868729, 0.02347051, 0.01940708, ..., 0.02067322, 0.01565038,
         0.01445595],
        [0.01290131, 0.01236447, 0.01458175, ..., 0.01160543, 0.01623518,
         0.0159892 ],
        [0.01354672, 0.02321601, 0.00816486, ..., 0.01915913, 0.01225124,
         0.01167687]]),
 'metric_means': array([0.01944555, 0.01752645, 0.02294285, 0.0151973 , 0.01745392,
        0.02138766, 0.02141948, 0.01730466, 0.01667541, 0.02010209,
        0.01861992, 0.02127924, 0.0175428 , 0.01989317, 0.01916358,
        0.01824808, 0.01788354, 0.01365811, 0.0184432 , 0.01649976,
        0.01548945, 0.01939683, 0.01750527, 0.01648026, 0.01919076,
        0.01501218, 0.02102495, 0.018

## Jetnet tests

### FPD Test

In [3]:
TwoSampleTestInputs = GMetrics.TwoSampleTestInputs(dist_1_input = dist_1,
                                                   dist_2_input = dist_2,
                                                   niter = 10,
                                                   batch_size_test = 10_000,
                                                   batch_size_gen = 100,
                                                   small_sample_threshold = 1e7,
                                                   dtype_input = tf.float64,
                                                   seed_input = 0,
                                                   use_tf = False,
                                                   mirror_strategy = False,
                                                   verbose = True)
print("nsamples",TwoSampleTestInputs.nsamples)
print("batch_size",TwoSampleTestInputs.batch_size_test)
print("niter",TwoSampleTestInputs.niter)
print("niter * batch_size",TwoSampleTestInputs.niter*TwoSampleTestInputs.batch_size_test)
print("small_sample",TwoSampleTestInputs.small_sample)

Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.
Using numpy mode with TensorFlow inputs.
Parsing input distribution...
Input distribution is a tfp.distributions.Distribution object.


Generating dist_1_num and dist_2_num with tensorflow function.
Checking and setting numerical distributions.
Setting dist_num from dist_symb.
Setting dist_num from dist_symb.
nsamples 100000
batch_size 10000
niter 10
niter * batch_size 100000
small_sample True


In [11]:
d1 = np.array(TwoSampleTestInputs.dist_1_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
d2 = np.array(TwoSampleTestInputs.dist_2_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
print("------------------------------------------")
print("Starting FPD metric calculation...")
print("Running Jetnet FPD calculation...")
print("niter = 10")
print("batch_size = 10000")
start = timer()
FPDResult = [JMetrics.fpd(s1,s2)[0] for s1,s2 in zip(d1,d2)]
end = timer()
print(f"FPD metric calculation completed in {end-start} seconds.")

------------------------------------------
Starting FPD metric calculation...
Running Jetnet FPD calculation...
niter = 10
batch_size = 10000


/local_data/scratch/rtorre/anaconda3/envs/tf2_9/lib/python3.8/site-packages/jetnet/evaluation/gen_metrics.py:698: RuntimeWarning: Recommended number of samples for FPD estimation is 50,000
  warnings.warn("Recommended number of samples for FPD estimation is 50,000", RuntimeWarning)


FPD metric calculation completed in 11.634189208038151 seconds.


In [ ]:
FPDResult

[0.00021711883420494262,
 0.0003158271980843811,
 0.00027361061888014936,
 0.00023580432171927144,
 0.00017307851172206515,
 0.00019975293044208664,
 0.000291297030758978,
 0.00023017760328604512,
 0.00018852383969245127,
 0.00017794826133951485]

### KPD Test

In [12]:
d1 = np.array(TwoSampleTestInputs.dist_1_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
d2 = np.array(TwoSampleTestInputs.dist_2_num).reshape(TwoSampleTestInputs.niter,TwoSampleTestInputs.batch_size_test,-1)
print("------------------------------------------")
print("Starting KPD metric calculation...")
print("Running Jetnet KPD calculation...")
print("niter = 10")
print("batch_size = 10000")
start = timer()
KPDResult = [JMetrics.kpd(s1,s2)[0] for s1,s2 in zip(d1,d2)]
end = timer()
print(f"KPD metric calculation completed in {end-start} seconds.")

------------------------------------------
Starting KPD metric calculation...
Running Jetnet KPD calculation...
niter = 10
batch_size = 10000


KeyboardInterrupt: 

In [ ]:
KPDResult

[0.10191366092886689,
 0.0799296299077179,
 0.11171410289918526,
 0.1536952109961831,
 -0.17934906441234943,
 -0.1042554083730014,
 0.09455262272575826,
 -0.12348942207734659,
 0.19918233408035757,
 0.15004449810881493]

In [4]:
from numpy.typing import ArrayLike
from numba import njit, prange, set_num_threads
from scipy.stats import iqr

X = np.array(TwoSampleTestInputs.dist_1_num)[:10_000]
Y = np.array(TwoSampleTestInputs.dist_2_num)[:10_000]

In [5]:
def _poly_kernel_pairwise(X: ArrayLike, Y: ArrayLike, degree: int) -> np.ndarray:
    """Pairwise polynomial kernel of degree ``degree`` between X and Y"""
    gamma = 1.0 / X.shape[-1]
    return (X @ Y.T * gamma + 1.0) ** degree

@njit
def _poly_kernel_pairwise_compiled(X: ArrayLike, Y: ArrayLike, degree: int) -> np.ndarray:
    """Pairwise polynomial kernel of degree ``degree`` between X and Y"""
    gamma = 1.0 / X.shape[-1]
    return (X @ Y.T * gamma + 1.0) ** degree

@tf.function(jit_compile=True)
def _poly_kernel_pairwise_tf(X, Y, degree):
    gamma = tf.cast(1.0, X.dtype) / tf.cast(tf.shape(X)[-1], X.dtype)
    return tf.pow(tf.linalg.matmul(X, Y, transpose_b=True) * gamma + 1.0, degree)

In [6]:
print(f"X[0] = {X[0]}")
print(f"Y[0] = {Y[0]}")
start = timer()
res_np = _poly_kernel_pairwise(X, Y, 4)
end = timer()
print(f"np result {res_np[0]} computed in {end-start} seconds.")
start = timer()
res_np_njit = _poly_kernel_pairwise_compiled(X, Y, 4)
end = timer()
print(f"np njit result {res_np_njit[0]} computed in {end-start} seconds.")
start = timer()
res_tf = _poly_kernel_pairwise_tf(X, Y, 4)
end = timer()
print(f"tf result {res_tf[0]} computed in {end-start} seconds.")

X[0] = [-0.23996704 -0.7940307   1.63931164  2.71145073 -0.90790552]
Y[0] = [-0.5684531  -2.14725651  0.06054185 -0.85779686  1.25558408]


np result [ 0.23326452  0.69533543  2.23488768 ...  1.09307324 12.7527977
  0.42137306] computed in 6.523499866016209 seconds.
np njit result [3.23384368e+03 4.19824216e+00 2.74021659e+03 ... 8.30164874e+00
 6.67361534e+05 6.36926289e+02] computed in 2.677459759172052 seconds.
tf result [ 0.23326452  0.69533543  2.23488768 ...  1.09307324 12.7527977
  0.42137306] computed in 5.172051236964762 seconds.


In [7]:
def _mmd_quadratic_unbiased(XX: ArrayLike, YY: ArrayLike, XY: ArrayLike):
    """Calculate quadratic estimate for MMD given pairwise distances between X and Y"""
    m, n = XX.shape[0], YY.shape[0]
    # subtract diagonal 1s
    return (
        (XX.sum() - np.trace(XX)) / (m * (m - 1))
        + (YY.sum() - np.trace(YY)) / (n * (n - 1))
        - 2 * np.mean(XY))
    
@njit
def _mmd_quadratic_unbiased_compiled(XX: ArrayLike, YY: ArrayLike, XY: ArrayLike):
    """Calculate quadratic estimate for MMD given pairwise distances between X and Y"""
    m, n = XX.shape[0], YY.shape[0]
    # subtract diagonal 1s
    return (
        (XX.sum() - np.trace(XX)) / (m * (m - 1))
        + (YY.sum() - np.trace(YY)) / (n * (n - 1))
        - 2 * np.mean(XY))
    
@tf.function(jit_compile=True)
def _mmd_quadratic_unbiased_tf(XX, YY, XY):
    m = tf.cast(tf.shape(XX)[0], XX.dtype)
    n = tf.cast(tf.shape(YY)[0], YY.dtype)
    return (tf.reduce_sum(XX) - tf.linalg.trace(XX)) / (m * (m - 1)) \
           + (tf.reduce_sum(YY) - tf.linalg.trace(YY)) / (n * (n - 1)) \
           - 2 * tf.reduce_mean(XY)

In [8]:
XX = _poly_kernel_pairwise(X, X, degree=2)
YY = _poly_kernel_pairwise(Y, Y, degree=2)
XY = _poly_kernel_pairwise(X, Y, degree=2)

In [9]:
print(f"XX[0] = {XX[0]}")
print(f"YY[0] = {YY[0]}")
print(f"XY[0] = {XY[0]}")
start = timer()
res_np = _mmd_quadratic_unbiased(XX, YY, XY)
end = timer()
print(f"np result {res_np} computed in {end-start} seconds.")
start = timer()
res_np_njit = _mmd_quadratic_unbiased_compiled(XX, YY, XY)
end = timer()
print(f"np njit result {res_np_njit} computed in {end-start} seconds.")
start = timer()
res_tf = _mmd_quadratic_unbiased_tf(XX, YY, XY)
end = timer()
print(f"tf result {res_tf} computed in {end-start} seconds.")

XX[0] = [1.09583100e+01 7.55214912e-01 1.16980924e-01 ... 1.35095661e-03
 2.50390433e-01 2.17284137e+00]
YY[0] = [6.00232518 1.92340716 1.66960703 ... 0.41960837 0.30236371 1.77563552]
XY[0] = [0.48297466 0.83386775 1.49495407 ... 1.04550143 3.57110595 0.64913254]
np result 0.00015766864370458933 computed in 0.2746531367301941 seconds.
np njit result 0.00015766864560129434 computed in 0.7537028132937849 seconds.
tf result 0.00015766864371880018 computed in 3.885398516897112 seconds.


In [10]:
def _mmd_poly_quadratic_unbiased(X: ArrayLike, Y: ArrayLike, degree: int = 4) -> float:
    """Calculate quadratic estimate for MMD with a polynomial kernel of degree ``degree``"""
    XX = _poly_kernel_pairwise(X, X, degree=degree)
    YY = _poly_kernel_pairwise(Y, Y, degree=degree)
    XY = _poly_kernel_pairwise(X, Y, degree=degree)
    return _mmd_quadratic_unbiased(XX, YY, XY)

@njit
def _mmd_poly_quadratic_unbiased_compiled(X: ArrayLike, Y: ArrayLike, degree: int = 4) -> float:
    """Calculate quadratic estimate for MMD with a polynomial kernel of degree ``degree``"""
    XX = _poly_kernel_pairwise_compiled(X, X, degree=degree)
    YY = _poly_kernel_pairwise_compiled(Y, Y, degree=degree)
    XY = _poly_kernel_pairwise_compiled(X, Y, degree=degree)
    return _mmd_quadratic_unbiased_compiled(XX, YY, XY)

@tf.function(jit_compile=True)
def _mmd_poly_quadratic_unbiased_tf(X, Y, degree=4):
    XX = _poly_kernel_pairwise_tf(X, X, degree=degree)
    YY = _poly_kernel_pairwise_tf(Y, Y, degree=degree)
    XY = _poly_kernel_pairwise_tf(X, Y, degree=degree)
    return _mmd_quadratic_unbiased_tf(XX, YY, XY)

In [11]:
print(f"X[0] = {X[0]}")
print(f"Y[0] = {Y[0]}")
start = timer()
res_np = _mmd_poly_quadratic_unbiased(X, Y, degree=4)
end = timer()
print(f"np result {res_np} computed in {end-start} seconds.")
start = timer() 
res_np_njit = _mmd_poly_quadratic_unbiased_compiled(X, Y, degree=4)
end = timer()
print(f"np njit result {res_np_njit} computed in {end-start} seconds.")
start = timer()
res_tf = _mmd_poly_quadratic_unbiased_tf(X, Y, degree=4)
end = timer()
print(f"tf result {res_tf} computed in {end-start} seconds.")

X[0] = [-0.23996704 -0.7940307   1.63931164  2.71145073 -0.90790552]
Y[0] = [-0.5684531  -2.14725651  0.06054185 -0.85779686  1.25558408]


np result 0.0008545656248477584 computed in 20.508854711893946 seconds.
np njit result -1301.0829068376916 computed in 7.2376785962842405 seconds.
tf result 0.0008545656248424294 computed in 1.020201823208481 seconds.


In [12]:
def _kpd_batches(X, Y, num_batches, batch_size, seed):
    vals_point = []
    for i in range(num_batches):
        np.random.seed(seed + i * 1_000)
        rand1 = np.random.choice(len(X), size=batch_size)
        rand2 = np.random.choice(len(Y), size=batch_size)

        rand_sample1 = X[rand1]
        rand_sample2 = Y[rand2]

        val = _mmd_poly_quadratic_unbiased(rand_sample1, rand_sample2)
        vals_point.append(val)

    return vals_point

@njit(parallel=True)
def _kpd_batches_parallel(X, Y, num_batches, batch_size, seed):
    vals_point = np.zeros(num_batches, dtype=np.float64)
    for i in prange(num_batches):
        np.random.seed(seed + i * 1000)  # in case of multi-threading
        rand1 = np.random.choice(len(X), size=batch_size)
        rand2 = np.random.choice(len(Y), size=batch_size)

        rand_sample1 = X[rand1]
        rand_sample2 = Y[rand2]

        val = _mmd_poly_quadratic_unbiased_compiled(rand_sample1, rand_sample2, degree=4)
        vals_point[i] = val

    return vals_point

@tf.function(jit_compile=True)
def _kpd_batches_tf(X, Y, num_batches, batch_size, seed):
    vals_point = []
    for i in range(num_batches):
        tf.random.set_seed(seed + i * 1000)
        rand1 = tf.random.uniform(shape=(batch_size,), minval=0, maxval=len(X), dtype=tf.int32)
        rand2 = tf.random.uniform(shape=(batch_size,), minval=0, maxval=len(Y), dtype=tf.int32)
        
        rand_sample1 = tf.gather(X, rand1)
        rand_sample2 = tf.gather(Y, rand2)

        val = _mmd_poly_quadratic_unbiased_tf(rand_sample1, rand_sample2)
        vals_point.append(val)
    vals_point = tf.stack(vals_point)
    return vals_point

In [23]:
print(f"X[0] = {X[0]}")
print(f"Y[0] = {Y[0]}")
start = timer()
res_np = _kpd_batches(X, Y, num_batches=10, batch_size=100, seed=0)
end = timer()
print(f"np result {res_np} computed in {end-start} seconds.")
start = timer()
res_np_njit = _kpd_batches_parallel(X, Y, num_batches=10, batch_size=100, seed=0)
end = timer()
print(f"np njit result {res_np_njit} computed in {end-start} seconds.")
start = timer()
res_tf = _kpd_batches_tf(X, Y, num_batches=10, batch_size=100, seed=0)
end = timer()
print(f"tf result {res_tf} computed in {end-start} seconds.")

X[0] = [-0.23996704 -0.7940307   1.63931164  2.71145073 -0.90790552]
Y[0] = [-0.5684531  -2.14725651  0.06054185 -0.85779686  1.25558408]
np result [-0.1254091521238685, 0.195738473673722, -0.027293622979322585, -0.07839786111071412, -0.06219736021454203, 0.11355592943788917, -0.11691377842527562, -0.18494873290895963, -0.16447388822740194, 0.010497979757301046] computed in 0.018750227987766266 seconds.


np njit result [-0.12540915  0.19573847 -0.02729362 -0.07839786 -0.06219736  0.11355593
 -0.11691378 -0.18494873 -0.16447389  0.01049798] computed in 3.244482960086316 seconds.
tf result [ 0.10614752 -0.14000339  0.10036664 -0.03384948  0.11692506 -0.0754364
 -0.10301249 -0.01171646 -0.02066491 -0.08897503] computed in 6.007333888672292 seconds.


In [24]:
def kpd(
    X_input: np.ndarray,
    Y_input: np.ndarray,
    num_batches: int = 10,
    batch_size: int = 5000,
    normalise: bool = True,
    seed: int = 42,
    num_threads: int = None,
    ) -> Tuple[float, float]:
    """Calculates the median and error of the kernel physics distance (KPD) between a set of real
    and generated features, as defined in https://arxiv.org/abs/2211.10295.

    It is recommended to use input sample sizes of at least 50,000, and the default values for other
    input parameters for consistency with other measurements.

    Similarly, for jets, it is recommended to use the set of EFPs as provided by the
    ``get_fpd_kpd_jet_features`` method.

    Args:
        real_features (Union[Tensor, np.ndarray]): set of real features of shape
          ``[num_samples, num_features]``.
        gen_features (Union[Tensor, np.ndarray]): set of generated features of shape
          ``[num_samples, num_features]``.
        num_batches (int, optional): number of batches to average over. Defaults to 10.
        batch_size (int, optional): size of each batch for which MMD is measured. Defaults to 5,000.
        normalise (bool, optional): normalise the individual features over the full sample to have
          the same scaling. Defaults to True.
        seed (int, optional): random seed. Defaults to 42.
        num_threads (int, optional): parallelize KPD through numba using this many threads. 0 means
          numba's default number of threads, based on # of cores available. Defaults to None, i.e.
          no parallelization.

    Returns:
        Tuple[float, float]: median and error of KPD.
    """
    
    if normalise:
        X, Y = _normalise_features(X_input, Y_input) # type: ignore
    else:
        X, Y = X_input, Y_input
    if num_threads is None:
        vals_point = _kpd_batches(X, Y, num_batches, batch_size, seed)
    else:
        if num_threads > 0:
            set_num_threads(num_threads)

        vals_point = _kpd_batches_parallel(X, Y, num_batches, batch_size, seed)

    # median, error = half of 16 - 84 IQR
    return (np.median(vals_point), iqr(vals_point, rng=(16.275, 83.725)) / 2)


def kpd_tf(
    X_input: tf.Tensor,
    Y_input: tf.Tensor,
    num_batches: int = 10,
    batch_size: int = 5000,
    normalise: bool = True,
    seed: int = 42,
    num_threads: int = None,
    ) -> (tf.Tensor, tf.Tensor):
    
    if normalise:
        X, Y = _normalise_features_tf(X_input, Y_input)  # Assume this is a TF function
    else:
        X, Y = X_input, Y_input
    
    if num_threads is None:
        vals_point = _kpd_batches_tf(X, Y, num_batches, batch_size, seed)  # Assume this is a TF function
    else:
        # TensorFlow handles multi-threading automatically, so we don't need to set threads manually.
        vals_point = _kpd_batches_tf(X, Y, num_batches, batch_size, seed)  # Assume this is a TF function
    
    # Calculating median and IQR using TensorFlow
    median = tfp.stats.percentile(vals_point, 50.0, axis=0)
    iqr_value = iqr_tf(vals_point, rng=(16.275, 83.725)) / 2  # Assume this is a TF function
    
    return (median, iqr_value)


In [26]:
d1 = np.array(TwoSampleTestInputs.dist_1_num)[:10_000]
d2 = np.array(TwoSampleTestInputs.dist_2_num)[:10_000]
print(f"d1.shape = {d1.shape}")
start = timer()
res_np_1 = JMetrics.kpd(d1,d2)
end = timer()
print(f"np result {res_np_1} computed in {end-start} seconds.")
start = timer()
res_np_2 = kpd(d1,d2)
end = timer()
print(f"np result {res_np_2} computed in {end-start} seconds.")
start = timer()
res_tf = kpd_tf(d1,d2)
end = timer()
print(f"tf result {res_tf} computed in {end-start} seconds.")

d1.shape = (10000, 5)
np result (-0.012937978482451484, 0.2632910392839428) computed in 16.325843856669962 seconds.


NameError: name '_normalise_features' is not defined